Validation notebook for statistical methods in the Gray Whale Project
====================================

Dependencies
-------------

In [1]:
%AddDeps org.bdgenomics.adam adam-core-spark2_2.11 0.24.0
%AddDeps comp.bio.aging adam-playground_2.11 0.0.13 --repository https://dl.bintray.com/comp-bio-aging/main/

Marking org.bdgenomics.adam:adam-core-spark2_2.11:0.24.0 for download
Preparing to fetch from:
-> file:/tmp/toree-tmp-dir561821172751950842/toree_add_deps/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree-tmp-dir561821172751950842/toree_add_deps/https/repo1.maven.org/maven2/org/bdgenomics/adam/adam-core-spark2_2.11/0.24.0/adam-core-spark2_2.11-0.24.0.jar
-> New file at /tmp/toree-tmp-dir561821172751950842/toree_add_deps/https/repo1.maven.org/maven2/org/bdgenomics/adam/adam-core-spark2_2.11/0.24.0/adam-core-spark2_2.11-0.24.0.pom


Waiting for a Spark session to start...

Marking comp.bio.aging:adam-playground_2.11:0.0.13 for download
Preparing to fetch from:
-> file:/tmp/toree-tmp-dir561821172751950842/toree_add_deps/
-> https://dl.bintray.com/comp-bio-aging/main/
-> https://repo1.maven.org/maven2


In [2]:
%AddDeps org.apache.hadoop hadoop-azure 2.7.6
%AddDeps com.microsoft.azure azure-storage 2.0.0

Marking org.apache.hadoop:hadoop-azure:2.7.6 for download
Preparing to fetch from:
-> file:/tmp/toree-tmp-dir561821172751950842/toree_add_deps/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree-tmp-dir561821172751950842/toree_add_deps/https/repo1.maven.org/maven2/org/apache/hadoop/hadoop-azure/2.7.6/hadoop-azure-2.7.6.jar
-> New file at /tmp/toree-tmp-dir561821172751950842/toree_add_deps/https/repo1.maven.org/maven2/org/apache/hadoop/hadoop-azure/2.7.6/hadoop-azure-2.7.6.pom


Waiting for a Spark session to start...

Marking com.microsoft.azure:azure-storage:2.0.0 for download
Preparing to fetch from:
-> file:/tmp/toree-tmp-dir561821172751950842/toree_add_deps/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree-tmp-dir561821172751950842/toree_add_deps/https/repo1.maven.org/maven2/com/microsoft/azure/azure-storage/2.0.0/azure-storage-2.0.0.jar
-> New file at /tmp/toree-tmp-dir561821172751950842/toree_add_deps/https/repo1.maven.org/maven2/com/microsoft/azure/azure-storage/2.0.0/azure-storage-2.0.0.pom


Imports and general functions
===============

In [3]:
import  org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.sql.types.StructType
import scala.reflect.runtime.universe._
import org.apache.spark.storage.StorageLevel
import org.apache.spark.rdd._
import org.apache.spark.ml.linalg._
import org.apache.spark.ml.stat._

In [4]:
//import comp.bio.aging.playground.extensions._
//import org.bdgenomics.adam.rdd.ADAMContext._

Name: Syntax Error.
Message: 
StackTrace: 

In [5]:
def sparkHadoopConf(sc: SparkContext, acountName: String, accountKey: String) = {
  sc.hadoopConfiguration.set("fs.azure", "org.apache.hadoop.fs.azure.NativeAzureFileSystem")
  sc.hadoopConfiguration.set("fs.azure.account.key." + acountName + ".blob.core.windows.net", accountKey)
  sc
}

sparkHadoopConf: (sc: org.apache.spark.SparkContext, acountName: String, accountKey: String)org.apache.spark.SparkContext


In [6]:
def azurize(container: String, accountName: String, blobFile: String): String = "wasbs://"+container+"@"+accountName+".blob.core.windows.net/"+blobFile 

azurize: (container: String, accountName: String, blobFile: String)String


In [7]:
val account = "pipelines1"
val key = ""
val connString = s"DefaultEndpointsProtocol=https;AccountName=pipelines1;AccountKey=${key};EndpointSuffix=core.windows.net"
def az(path: String): String = azurize("storage", account, path)

account = pipelines1
key = 
connString = DefaultEndpointsProtocol=https;AccountName=pipelines1;AccountKey=;EndpointSuffix=core.windows.net


az: (path: String)String


DefaultEndpointsProtocol=https;AccountName=pipelines1;AccountKey=;EndpointSuffix=core.windows.net

In [8]:
def write(df: DataFrame, url: String, header: Boolean = true) = {
  df.coalesce(1).write.option("sep","\t").option("header", header).csv(url)
  url
}

write: (df: org.apache.spark.sql.DataFrame, url: String, header: Boolean)String


In [9]:
val sparkContext = sc
sparkHadoopConf(sparkContext, account, key)
  
val spark = SparkSession
  .builder()
  .appName("whales_validation")
  .getOrCreate()

sparkContext = org.apache.spark.SparkContext@77dd51fb
spark = org.apache.spark.sql.SparkSession@48a08343


Spark app-20180703125720-0000: Some(http://c8fc1f462c1d:4040)

In [23]:
import org.apache.spark.sql.functions._
import spark.implicits._

val toDouble = udf[Double, String]( _.toDouble)

toDouble = UserDefinedFunction(<function1>,DoubleType,Some(List(StringType)))


UserDefinedFunction(<function1>,DoubleType,Some(List(StringType)))

In [24]:
def readTSV(path: String, header: Boolean = false, sep: String = "\t", infer: Boolean = true): DataFrame = spark.read
    .option("sep", sep)
    .option("comment", "#")
    .option("ignoreLeadingWhiteSpace", true)
    .option("ignoreTrailingWhiteSpace", true)
    .option("ignoreTrailingWhiteSpace", true)
    .option("inferSchema", infer)
    .option("header", header)
    .option("maxColumns", 150000)
    .csv(path)

readTSV: (path: String, header: Boolean, sep: String, infer: Boolean)org.apache.spark.sql.DataFrame


In [25]:
def readTypedTSV[T <: Product](path: String, header: Boolean = false, sep: String = "\t")
                              (implicit tag: TypeTag[T]): Dataset[T] = {
  implicit val encoder: StructType = Encoders.product[T](tag).schema
  spark.read
    .option("sep", sep)
    .option("comment", "#")
    .option("ignoreLeadingWhiteSpace", true)
    .option("ignoreTrailingWhiteSpace", true)
    .option("ignoreTrailingWhiteSpace", true)
    .option("header", header)
    .schema(encoder)
    .csv(path).as[T]
}

readTypedTSV: [T <: Product](path: String, header: Boolean, sep: String)(implicit tag: reflect.runtime.universe.TypeTag[T])org.apache.spark.sql.Dataset[T]


In [26]:
 def toVectors(dataFrame: DataFrame, columns: Seq[String], output: String) = {  
      import org.apache.spark.ml.feature.VectorAssembler
      import org.apache.spark.ml.linalg.Vectors

      val assembler = new VectorAssembler()
        .setInputCols(columns.toArray)
        .setOutputCol(output)

      assembler.transform(dataFrame.na.fill(0.0, columns).na.fill("")).select(output)
    }

toVectors: (dataFrame: org.apache.spark.sql.DataFrame, columns: Seq[String], output: String)org.apache.spark.sql.DataFrame


Core code
=====

Loading pathes
---------------

In [55]:
val expressionsPath = "expressions"
val byGoPath = expressionsPath + "/go"
val comparison = byGoPath + "/gray_whale_with_bowhead_with_minke_with_NMR_with_human_with_mouse_with_cow_full_outer_counts_extended.tsv"
val grouped = byGoPath + "/grouped"
val ranked = byGoPath + "/grouped/ranked"
val transcriptsPath = expressionsPath + "/transcripts"

val indexesPath = "indexes"
val reactomePath = indexesPath + "/reactome"
val unirefPath = expressionsPath + "/" + "uniref90"

expressionsPath = expressions
byGoPath = expressions/go
comparison = expressions/go/gray_whale_with_bowhead_with_minke_with_NMR_with_human_with_mouse_with_cow_full_outer_counts_extended.tsv
grouped = expressions/go/grouped
ranked = expressions/go/grouped/ranked
transcriptsPath = expressions/transcripts
indexesPath = indexes
reactomePath = indexes/reactome
unirefPath = expressions/uniref90


expressions/uniref90

In [28]:
val goTable = readTSV(az(comparison), true)

goTable = [go: string, namespace: string ... 27 more fields]


[go: string, namespace: string ... 27 more fields]

In [29]:
%%dataframe goTable

go,namespace,label,gray_whale_avg,uniref90_count,taxons_count,transcripts_count,human_transcripts_count,mouse_transcripts_count,cow_transcripts_count,type,gray_whale_liver,bowhead_whale_liver,minke_liver,NMR_liver,human_liver,mouse_totalRNA_liver,mouse_mRNA_liver,cow_totalRNA_liver,cow_mRNA_liver,gray_whale_kidney,bowhead_whale_kidney,minke_kidney,NMR_kidney,human_kidney,mouse_totalRNA_kidney,mouse_mRNA_kidney,cow_totalRNA_kidney,cow_mRNA_kidney
GO:0070062,cellular_component,extracellular exosome,95231.76295,726,929,876,2714,116,2162,complete;5prime_partial;internal;3prime_partial,82627.97828900004,43406.81879601012,103879.82613499998,189153.98861699994,34179.85539700001,63059.86755299999,12146.704848999996,407115.5446490002,402875.4094160001,107835.54761100012,12291.983165,139226.62839100015,251177.67484400002,17317.253403999995,8189.928592999998,3009.2303389999997,284041.33320799994,274118.5267729999
GO:0016021,cellular_component,integral component of membrane,81241.0623675,1391,565,1713,9561,8900,4715,5prime_partial;complete;3prime_partial;internal,62374.28512099997,27353.04877027518,55178.75498600001,108836.22230300008,11032.347553,168821.304916,28909.156348999997,198371.61491800006,189068.7378029999,100107.839614,24343.312661999982,94079.22781100006,149277.02259700003,30508.606157000006,241562.48318399998,51496.39662599999,315467.63018200017,373243.68564099993
GO:0003735,molecular_function,structural constituent of ribosome,61936.161357499994,98,380,107,467,306,261,complete;5prime_partial;3prime_partial;internal,62265.27571699998,21665.888836018064,62369.073224,103095.90135300001,5244.261549,24249.30922799999,7137.034891999996,89084.343089,87166.34631000001,61607.046997999976,8349.420524000001,88459.54913499996,70943.44369100004,4565.115427999999,50372.43014700001,11549.923136999998,94610.67463199992,72399.43132700001
GO:0005829,cellular_component,cytosol,61398.1790675,1109,994,1376,6775,5043,2348,complete;3prime_partial;5prime_partial;internal,50912.655819000014,29665.787975965563,72109.60757099999,188757.6132710001,20623.12592999999,148373.97883200005,100424.52617,86184.9085600001,90951.61626199998,71883.70231600002,12335.804511999993,124936.20551199997,132241.5113150001,13435.613476000004,105758.17169699995,43636.518504000014,111689.96638100002,102744.94863500002
GO:0005840,cellular_component,ribosome,59087.688684,89,316,99,297,123,40,complete;5prime_partial;internal;3prime_partial,58631.94472899998,23801.48132475636,65881.81530099995,103165.58999999998,2209.9476119999995,5478.616020999998,1831.119411,5936.298134999999,6073.805264,59543.432639000006,9270.050664999997,92530.64897999998,71492.44906799999,1924.4887119999999,10029.277271,2800.6401849999993,5875.225427000001,4481.681578999999
GO:0006412,biological_process,translation,58622.12868799999,105,377,116,468,286,193,complete;5prime_partial;3prime_partial;internal,58727.85268499997,20034.802301962056,58984.534049,99321.06117499995,5245.888943999999,19875.159683999995,5806.578341,66453.233819,66083.20671399997,58516.40469099998,7580.873805999998,83876.08076099999,68055.43613499998,4572.8019749999985,40136.835409,9191.198294,69545.42429499998,53565.293888999986
GO:0005634,cellular_component,nucleus,57345.87068550001,1431,2115,1830,7301,6496,2778,complete;5prime_partial;3prime_partial;internal,55747.40112099999,30295.434521280793,70441.20552599998,140240.092983,10470.131235999997,94231.85965399996,84031.996854,126193.50819700002,143407.437234,58944.34024999998,11416.312302999997,102915.768411,90533.17986400002,8808.929934000003,81393.57837199996,30863.47935199999,118101.15175700001,99420.20529599997
GO:0003723,molecular_function,RNA binding,54252.262738,621,368,821,2148,1069,1008,complete;5prime_partial;3prime_partial;internal,48843.76697,23928.740201085533,56334.43345600005,91851.60494800007,6522.695184999994,14026.664751000004,4708.064785000002,74843.61990199998,82199.186156,59660.75850600002,10025.436343999996,93364.80089000006,79318.97048099998,6672.883201000003,24

In [30]:
val features = List("gray_whale_liver", "bowhead_whale_liver", "minke_liver", "NMR_liver", "human_liver", "mouse_totalRNA_liver", "mouse_mRNA_liver", "cow_totalRNA_liver", "cow_mRNA_liver", "gray_whale_kidney", "bowhead_whale_kidney", "minke_kidney", "NMR_kidney", "human_kidney", "mouse_totalRNA_kidney", "mouse_mRNA_kidney", "cow_totalRNA_kidney", "cow_mRNA_kidney")

features = List(gray_whale_liver, bowhead_whale_liver, minke_liver, NMR_liver, human_liver, mouse_totalRNA_liver, mouse_mRNA_liver, cow_totalRNA_liver, cow_mRNA_liver, gray_whale_kidney, bowhead_whale_kidney, minke_kidney, NMR_kidney, human_kidney, mouse_totalRNA_kidney, mouse_mRNA_kidney, cow_totalRNA_kidney, cow_mRNA_kidney)


List(gray_whale_liver, bowhead_whale_liver, minke_liver, NMR_liver, human_liver, mouse_totalRNA_liver, mouse_mRNA_liver, cow_totalRNA_liver, cow_mRNA_liver, gray_whale_kidney, bowhead_whale_kidney, minke_kidney, NMR_kidney, human_kidney, mouse_totalRNA_kidney, mouse_mRNA_kidney, cow_totalRNA_kidney, cow_mRNA_kidney)

In [34]:
val vec = toVectors(goTable, features, "features").cache
vec

vec = [features: vector]


[features: vector]

In [32]:
%%dataframe vec

features
"[82627.97828900004,43406.81879601012,103879.82613499998,189153.98861699994,34179.85539700001,63059.86755299999,12146.704848999996,407115.5446490002,402875.4094160001,107835.54761100012,12291.983165,139226.62839100015,251177.67484400002,17317.253403999995,8189.928592999998,3009.2303389999997,284041.33320799994,274118.5267729999]"
"[62374.28512099997,27353.04877027518,55178.75498600001,108836.22230300008,11032.347553,168821.304916,28909.156348999997,198371.61491800006,189068.7378029999,100107.839614,24343.312661999982,94079.22781100006,149277.02259700003,30508.606157000006,241562.48318399998,51496.39662599999,315467.63018200017,373243.68564099993]"
"[62265.27571699998,21665.888836018064,62369.073224,103095.90135300001,5244.261549,24249.30922799999,7137.034891999996,89084.343089,87166.34631000001,61607.046997999976,8349.420524000001,88459.54913499996,70943.44369100004,4565.115427999999,50372.43014700001,11549.923136999998,94610.67463199992,72399.43132700001]"
"[50912.655819000014,29665.787975965563,72109.60757099999,188757.6132710001,20623.12592999999,148373.97883200005,100424.52617,86184.9085600001,90951.61626199998,71883.70231600002,12335.804511999993,124936.20551199997,132241.5113150001,13435.613476000004,105758.17169699995,43636.518504000014,111689.96638100002,102744.94863500002]"
"[58631.94472899998,23801.48132475636,65881.81530099995,103165.58999999998,2209.9476119999995,5478.616020999998,1831.119411,5936.298134999999,6073.805264,59543.432639000006,9270.050664999997,92530.64897999998,71492.44906799999,1924.4887119999999,10029.277271,2800.6401849999993,5875.225427000001,4481.681578999999]"
"[58727.85268499997,20034.802301962056,58984.534049,99321.06117499995,5245.888943999999,19875.159683999995,5806.578341,66453.233819,66083.20671399997,58516.40469099998,7580.873805999998,83876.08076099999,68055.43613499998,4572.8019749999985,40136.835409,9191.198294,69545.42429499998,53565.293888999986]"
"[55747.40112099999,30295.434521280793,70441.20552599998,140240.092983,10470.131235999997,94231.85965399996,84031.996854,126193.50819700002,143407.437234,58944.34024999998,11416.312302999997,102915.768411,90533.17986400002,8808.929934000003,81393.57837199996,30863.47935199999,118101.15175700001,99420.20529599997]"
"[48843.76697,23928.740201085533,56334.43345600005,91851.60494800007,6522.695184999994,14026.664751000004,4708.064785000002,74843.61990199998,82199.186156,59660.75850600002,10025.436343999996,93364.80089000006,79318.97048099998,6672.883201000003,24822.313444999996,8102.410284000003,93044.76531800005,83947.353972]"
"[43367.513537999985,24393.98329442446,63289.08200100004,177635.51962800007,6716.158039999998,135388.78987699997,34234.04624200001,79546.89437599998,85249.95693999997,58676.24096099999,10090.376369000003,102810.643446,85731.96628100002,7004.031145,87788.121675,34874.35143699998,101489.413696,87811.42796300001]"
"[24852.939731,1933.4432743038858,4332.0924300000015,5095.546241,292.3133429999999,3911.2582859999984,1290.271993,6743.0331620000015,7806.252158000002,60057.036932000025,839.858576,9289.642984999997,3992.193603,583.6687029999998,7800.484518000001,3703.0777580000004,12280.558181999999,11146.476477]"


In [33]:
val cor =   Correlation.corr(vec, "features")

cor = [pearson(features): matrix]


[pearson(features): matrix]

Splitting by GO namespaces
------------------------------

In [35]:
val components = goTable.where("namespace = 'cellular_component'")
val processes = goTable.where("namespace = 'biological_process'")
val functions = goTable.where("namespace = 'molecular_function'")

components = [go: string, namespace: string ... 27 more fields]
processes = [go: string, namespace: string ... 27 more fields]
functions = [go: string, namespace: string ... 27 more fields]


[go: string, namespace: string ... 27 more fields]

In [36]:
(components.count, processes.count, functions.count, goTable.count)

(1739,12369,4396,18597)

In [33]:
//write(components, az(grouped + "/by_cellular_component.tsv"))
//write(processes, az(grouped + "/by_biological_process.tsv"))
//write(functions, az(grouped + "/by_molecular_function.tsv"))

Name: Syntax Error.
Message: 
StackTrace: 

Ranks computations
==================

In [38]:
import org.apache.spark.sql.{ColumnName, Dataset}
import org.apache.spark.sql.expressions.Window

def rank(df: DataFrame, name: String, rankSuffix: String = "_rank") =
  df.withColumn(name + rankSuffix, org.apache.spark.sql.functions.dense_rank().over(Window.orderBy(new ColumnName(name).desc)))

def ranks(df: DataFrame,
          names: Seq[String],
          rankSuffix: String = "_rank") = names.foldLeft(df){
  case (f, n)=> rank(f, n, rankSuffix)
}

rank: (df: org.apache.spark.sql.DataFrame, name: String, rankSuffix: String)org.apache.spark.sql.DataFrame
ranks: (df: org.apache.spark.sql.DataFrame, names: Seq[String], rankSuffix: String)org.apache.spark.sql.DataFrame


In [39]:
val rankedCols = components.columns.drop(11)++List("gray_whale_avg")

Name: Unknown Error
Message: <console>:225: error: not found: value components
       val rankedCols = components.columns.drop(11)++List("gray_whale_avg")
                        ^

StackTrace: 

In [41]:
val rankedComponents = ranks(components, rankedCols)
val rankedProcesses = ranks(processes, rankedCols)
val rankedFunctions = ranks(functions, rankedCols)

rankedComponents = [go: string, namespace: string ... 46 more fields]
rankedProcesses = [go: string, namespace: string ... 46 more fields]
rankedFunctions = [go: string, namespace: string ... 46 more fields]


[go: string, namespace: string ... 46 more fields]

In [3]:
write(rankedComponents, az(grouped + "/by_cellular_component_ranked.tsv"))
write(rankedProcesses, az(grouped + "/by_biological_process_ranked.tsv"))
write(rankedFunctions, az(grouped + "/by_molecular_function_ranked.tsv"))

wasbs://storage@pipelines1.blob.core.windows.net/expressions/go/grouped/by_molecular_function_ranked.tsv

In [26]:
%%dataframe rankedComponents

go,namespace,label,gray_whale_avg,uniref90_count,taxons_count,transcripts_count,human_transcripts_count,mouse_transcripts_count,cow_transcripts_count,type,gray_whale_liver,bowhead_whale_liver,minke_liver,NMR_liver,human_liver,mouse_totalRNA_liver,mouse_mRNA_liver,cow_totalRNA_liver,cow_mRNA_liver,gray_whale_kidney,bowhead_whale_kidney,minke_kidney,NMR_kidney,human_kidney,mouse_totalRNA_kidney,mouse_mRNA_kidney,cow_totalRNA_kidney,cow_mRNA_kidney,gray_whale_liver_rank,bowhead_whale_liver_rank,minke_liver_rank,NMR_liver_rank,human_liver_rank,mouse_totalRNA_liver_rank,mouse_mRNA_liver_rank,cow_totalRNA_liver_rank,cow_mRNA_liver_rank,gray_whale_kidney_rank,bowhead_whale_kidney_rank,minke_kidney_rank,NMR_kidney_rank,human_kidney_rank,mouse_totalRNA_kidney_rank,mouse_mRNA_kidney_rank,cow_totalRNA_kidney_rank,cow_mRNA_kidney_rank,gray_whale_avg_rank
GO:0070062,cellular_component,extracellular exosome,95231.76295,726,929,876,2714,116,2162,complete;5prime_partial;internal;3prime_partial,82627.97828900004,43406.81879601012,103879.82613499998,189153.98861699994,34179.85539700001,63059.86755299999,12146.704848999996,407115.5446490002,402875.4094160001,107835.54761100012,12291.983165,139226.62839100015,251177.67484400002,17317.253403999995,8189.928592999998,3009.2303389999997,284041.33320799994,274118.5267729999,1,1,1,1,1,12,16,1,1,1,4,1,1,3,46,45,2,2,1
GO:0016021,cellular_component,integral component of membrane,81241.0623675,1391,565,1713,9561,8900,4715,5prime_partial;complete;3prime_partial;internal,62374.28512099997,27353.04877027518,55178.75498600001,108836.22230300008,11032.347553,168821.304916,28909.156348999997,198371.61491800006,189068.7378029999,100107.839614,24343.312661999982,94079.22781100006,149277.02259700003,30508.606157000006,241562.48318399998,51496.39662599999,315467.63018200017,373243.68564099993,2,4,6,6,8,3,8,2,2,2,1,5,2,1,2,2,1,1,2
GO:0005829,cellular_component,cytosol,61398.1790675,1109,994,1376,6775,5043,2348,complete;3prime_partial;5prime_partial;internal,50912.655819000014,29665.787975965563,72109.60757099999,188757.6132710001,20623.12592999999,148373.97883200005,100424.52617,86184.9085600001,90951.61626199998,71883.70231600002,12335.804511999993,124936.20551199997,132241.5113150001,13435.613476000004,105758.17169699995,43636.518504000014,111689.96638100002,102744.94863500002,5,3,2,2,4,5,2,5,6,3,3,2,3,4,3,3,4,4,3
GO:0005840,cellular_component,ribosome,59087.688684,89,316,99,297,123,40,complete;5prime_partial;internal;3prime_partial,58631.94472899998,23801.48132475636,65881.81530099995,103165.58999999998,2209.9476119999995,5478.616020999998,1831.119411,5936.298134999999,6073.805264,59543.432639000006,9270.050664999997,92530.64897999998,71492.44906799999,1924.4887119999999,10029.277271,2800.6401849999993,5875.225427000001,4481.681578999999,3,6,4,7,37,71,70,69,71,5,7,6,7,24,42,46,63,71,4
GO:0005634,cellular_component,nucleus,57345.87068550001,1431,2115,1830,7301,6496,2778,complete;5prime_partial;3prime_partial;internal,55747.40112099999,30295.434521280793,70441.20552599998,140240.092983,10470.131235999997,94231.85965399996,84031.996854,126193.50819700002,143407.437234,58944.34024999998,11416.312302999997,102915.768411,90533.17986400002,8808.929934000003,81393.57837199996,30863.47935199999,118101.15175700001,99420.20529599997,4,2,3,4,10,9,3,4,4,6,5,3,4,9,7,5,3,5,5
GO:0005737,cellular_component,cytoplasm,51021.87724949999,1070,1897,1342,4490,5424,2333,complete;5prime_partial;3prime_partial;internal,43367.513537999985,24393.98329442446,63289.08200100004,177635.51962800007,6716.158039999998,135388.78987699997,34234.04624200001,79546.89437599998,85249.95693999997,58676.24096099999,10090.376369000003,102810.643446,85731.96628100002,7004.031145,87788.121675,34874.35143699998,101489.413696,87811.42796300001,6,5,5,3,19,6,6,7,7,7,6,4,5,12,5,4,5,8,6
GO:0005813,cellular_component,centrosome,42454.98833149999,127,243,162,632,769,382,complete;5prime_partial;internal;3prime_partial,24852.939731,1933.4432743038858,4332.0924300000015

Loading mappings
================

Here we load uniprot mapping for further usage.

In [16]:
import org.apache.spark.storage._

val mapping =  readTSV(az("/indexes/uniprot/idmapping_selected.tab"))
.toDF("uniprot_ac","uniprot_id","entrez","refSeq", "gi","pdb", "go", "uniref100", "uniref90", "uniref50", "uniparc", "pir", "taxon", "mim", "unigene", "pubmed", "embl", "embl_cds", "ensembl", "ensembl_trs", "ensembl_pro", "additional_pubmed")
.persist(StorageLevel.MEMORY_AND_DISK) 

mapping = [uniprot_ac: string, uniprot_id: string ... 20 more fields]


[uniprot_ac: string, uniprot_id: string ... 20 more fields]

In [26]:
%%dataframe mapping

uniprot_ac,uniprot_id,entrez,refSeq,gi,pdb,go,uniref100,uniref90,uniref50,uniparc,pir,taxon,mim,unigene,pubmed,embl,embl_cds,ensembl,ensembl_trs,ensembl_pro,additional_pubmed
Q6GZX4,001R_FRG3G,2947773,YP_031579.1,81941549; 49237298,null,GO:0006355; GO:0046782; GO:0006351,UniRef100_Q6GZX4,UniRef90_Q6GZX4,UniRef50_Q6GZX4,UPI00003B0FD4,null,654924,null,null,15165820,AY548484,AAT09660.1,null,null,null,null
Q6GZX3,002L_FRG3G,2947774,YP_031580.1,49237299; 81941548; 47060117,null,GO:0033644; GO:0016021,UniRef100_Q6GZX3,UniRef90_Q6GZX3,UniRef50_Q6GZX3,UPI00003B0FD5,null,654924,null,null,15165820,AY548484,AAT09661.1,null,null,null,null
Q197F8,002R_IIV3,4156251,YP_654574.1,109287880; 123808694; 106073503,null,null,UniRef100_Q197F8,UniRef90_Q197F8,UniRef50_Q197F8,UPI0000D83464,null,345201,null,null,16912294,DQ643392,ABF82032.1,null,null,null,null
Q197F7,003L_IIV3,4156252,YP_654575.1,106073504; 109287881; 123808693,null,null,UniRef100_Q197F7,UniRef90_Q197F7,UniRef50_Q197F7,UPI0000D83465,null,345201,null,null,16912294,DQ643392,ABF82033.1,null,null,null,null
Q6GZX2,003R_FRG3G,2947775,YP_031581.1,47060118; 81941547; 49237300,null,null,UniRef100_Q6GZX2,UniRef90_Q6GZX2,UniRef50_Q6GZX2,UPI00003B0FD6,null,654924,null,null,15165820,AY548484,AAT09662.1,null,null,null,null
Q6GZX1,004R_FRG3G,2947776,YP_031582.1,49237301; 81941546,null,GO:0033644; GO:0016021,UniRef100_Q6GZX1,UniRef90_Q6GZX1,UniRef50_Q6GZX1,UPI00003B0FD7,null,654924,null,null,15165820,AY548484,AAT09663.1,null,null,null,null
Q197F5,005L_IIV3,4156254,YP_654577.1,123808691; 109287883; 106073506,null,null,UniRef100_Q197F5,UniRef90_Q197F5,UniRef50_Q197F5,UPI0000D83467,null,345201,null,null,16912294,DQ643392,ABF82035.1,null,null,null,null
Q6GZX0,005R_FRG3G,2947777,YP_031583.1,47060120; 49237302; 81941545,null,null,UniRef100_Q6GZX0,UniRef90_Q6GZX0,UniRef50_Q6GZX0,UPI00003B0FD8,null,654924,null,null,15165820,AY548484,AAT09664.1,null,null,null,null
Q91G88,006L_IIV6,1733056,NP_149469.1,82012333; 15078718; 15042163,null,null,UniRef100_Q91G88,UniRef90_Q91G88,UniRef50_Q91G88,UPI00000F3D32,null,176652,null,null,11448171; 17239238,AF303741,AAK81943.1,null,null,null,null
Q6GZW9,006R_FRG3G,2947778,YP_031584.1,81941544; 49237303,null,null,UniRef100_Q6GZW9,UniRef90_Q6GZW9,UniRef50_Q6GZW9,UPI00003B0FD9,null,654924,null,null,15165820,AY548484,AAT09665.1,null,null,null,null


In [76]:
val human_tax_id = "9606"
val mouse_tax_id = "10090"
val cow_tax_id = "9913"

val mapping_headers = Seq("uniprot_ac","uniprot_id","entrez","refSeq", "gi","pdb", "go", "uniref100", "uniref90", "uniref50", "uniparc", "pir", "taxon", "mim", "unigene", "pubmed", "embl", "embl_cds", "ensembl", "ensembl_trs", "ensembl_pro", "additional_pubmed")
  

val ind = az("/indexes/uniprot")
val human_mapping = readTSV(ind + "/HUMAN_9606_idmapping_selected.tab").toDF(mapping_headers:_*)
val mouse_mapping = readTSV(ind + "/MOUSE_10090_idmapping_selected.tab").toDF(mapping_headers:_*)
val cow_mapping = readTSV(ind + s"/COW_${cow_tax_id}_idmapping_selected.tab").toDF(mapping_headers:_*)

human_tax_id = 9606
mouse_tax_id = 10090
cow_tax_id = 9913
mapping_headers = List(uniprot_ac, uniprot_id, entrez, refSeq, gi, pdb, go, uniref100, uniref90, uniref50, uniparc, pir, taxon, mim, unigene, pubmed, embl, embl_cds, ensembl, ensembl_trs, ensembl_pro, additional_pubmed)
ind = wasbs://storage@pipelines1.blob.core.windows.net//indexes/uniprot
human_mapping = [uniprot_ac: string, uniprot_id: string ... 20 more fields]
mouse_mapping = [uniprot_ac: string, uniprot_id: string ... 20 more fields]
cow_mapping = [uniprot_ac: string, uniprot_id: string ... 20 more fields]


[uniprot_ac: string, uniprot_id: string ... 20 more fields]

In [95]:
def simplifyFunction = udf[String, String]{case tran =>
  val i = tran.indexOf('|')
  val tr = if(i> 0) tran.substring(0, tran.indexOf('|')) else tran
  tr.substring(0, Math.min(tr.indexOf('.'),tr.length))
}

simplifyFunction: org.apache.spark.sql.expressions.UserDefinedFunction


Correlations computations
==============

Here we compute correlations

Correlation functions
------------------------

In [36]:
def convertCorrellationMatrix(matrix: Matrix, columns: Seq[String]) = {
  require(columns.size == matrix.numCols)
  for(r <- 0 until matrix.numRows) yield {
    val seq = for(c <- 0 until matrix.numCols) yield matrix(r, c)
    Row.fromSeq(columns(r)::seq.toList)
  }
}

import org.apache.spark.sql.types._

def doublesByColumns(columns: Seq[String]) = columns.map(c=>StructField(c, DoubleType, false)).toList

def transformCorrellationMatrix(dataFrame: DataFrame, columns: Seq[String])(implicit sparkSession: SparkSession) = {
  import sparkSession.implicits._
  val rows  = dataFrame.rdd
    .flatMap{ case Row(matrix: Matrix) => convertCorrellationMatrix(matrix, columns) }
  sparkSession.createDataFrame(rows, StructType(StructField("feature", StringType, false)::doublesByColumns(columns)))
}

convertCorrellationMatrix: (matrix: org.apache.spark.ml.linalg.Matrix, columns: Seq[String])scala.collection.immutable.IndexedSeq[org.apache.spark.sql.Row]
doublesByColumns: (columns: Seq[String])List[org.apache.spark.sql.types.StructField]
transformCorrellationMatrix: (dataFrame: org.apache.spark.sql.DataFrame, columns: Seq[String])(implicit sparkSession: org.apache.spark.sql.SparkSession)org.apache.spark.sql.DataFrame


In [37]:
def pearsonCorrellation(dataFrame: DataFrame, columns: Seq[String])(implicit sparkSession: SparkSession) = {
  val cor = toVectors(dataFrame, columns, "features")
  val df = Correlation.corr(cor, "features")
  transformCorrellationMatrix(df, columns)
}

pearsonCorrellation: (dataFrame: org.apache.spark.sql.DataFrame, columns: Seq[String])(implicit sparkSession: org.apache.spark.sql.SparkSession)org.apache.spark.sql.DataFrame


In [38]:
def spearmanCorrellation(dataFrame: DataFrame, columns: Seq[String])(implicit sparkSession: SparkSession) = {
  val cor = toVectors(dataFrame, columns.toSeq, "features").persist(StorageLevel.MEMORY_AND_DISK)
  import org.apache.spark.ml.linalg.Matrix
  val df = Correlation.corr(cor, "features", method = "spearman")
  transformCorrellationMatrix(df, columns)
}

spearmanCorrellation: (dataFrame: org.apache.spark.sql.DataFrame, columns: Seq[String])(implicit sparkSession: org.apache.spark.sql.SparkSession)org.apache.spark.sql.DataFrame


Computing correlations to our data
---------------------------------------

In [4]:
val corPath = grouped + "/correlations"
val statsPath = byGoPath + "/stats"

corPath = expressions/go/grouped/correlations
statsPath = expressions/go/stats


expressions/go/stats

In [40]:
val p = pearsonCorrellation(goTable, features)(spark)

p = [feature: string, gray_whale_liver: double ... 17 more fields]


[feature: string, gray_whale_liver: double ... 17 more fields]

In [44]:
//write(p, az(corPath + "/pearson_go.tsv"))

wasbs://storage@pipelines1.blob.core.windows.net/expressions/go/grouped/correlations/pearson_go.tsv

In [41]:
val pCorComponents = pearsonCorrellation(components, features)(spark)
val pCorFunctions =  pearsonCorrellation(functions, features)(spark)
val pCorProcesses = pearsonCorrellation(processes, features)(spark)

pCorComponents = [feature: string, gray_whale_liver: double ... 17 more fields]
pCorFunctions = [feature: string, gray_whale_liver: double ... 17 more fields]
pCorProcesses = [feature: string, gray_whale_liver: double ... 17 more fields]


[feature: string, gray_whale_liver: double ... 17 more fields]

In [42]:
//write(pCorComponents, az(corPath + "/pearson_components.tsv"))
//write(pCorFunctions, az(corPath + "/pearson_functions.tsv"))
//write(pCorProcesses, az(corPath + "/pearson_proccesses.tsv"))

Name: Syntax Error.
Message: 
StackTrace: 

In [43]:
val s =  spearmanCorrellation(goTable, features)(spark)

s = [feature: string, gray_whale_liver: double ... 17 more fields]


[feature: string, gray_whale_liver: double ... 17 more fields]

In [44]:
//write(s, az(corPath + "/spearman_go.tsv"))

Name: Syntax Error.
Message: 
StackTrace: 

In [45]:
val sCorComponents = spearmanCorrellation(components, features)(spark)
val sCorFunctions =  spearmanCorrellation(functions, features)(spark)
val sCorProcesses = spearmanCorrellation(processes, features)(spark)

sCorComponents = [feature: string, gray_whale_liver: double ... 17 more fields]
sCorFunctions = [feature: string, gray_whale_liver: double ... 17 more fields]
sCorProcesses = [feature: string, gray_whale_liver: double ... 17 more fields]


[feature: string, gray_whale_liver: double ... 17 more fields]

In [46]:
//write(sCorComponents, az(corPath + "/spearman_components.tsv"))
//write(sCorFunctions, az(corPath + "/spearman_functions.tsv"))
//write(sCorProcesses, az(corPath + "/spearman_proccesses.tsv"))

Name: Syntax Error.
Message: 
StackTrace: 

In [47]:
%%dataframe p

feature,gray_whale_liver,bowhead_whale_liver,minke_liver,NMR_liver,human_liver,mouse_totalRNA_liver,mouse_mRNA_liver,cow_totalRNA_liver,cow_mRNA_liver,gray_whale_kidney,bowhead_whale_kidney,minke_kidney,NMR_kidney,human_kidney,mouse_totalRNA_kidney,mouse_mRNA_kidney,cow_totalRNA_kidney,cow_mRNA_kidney
gray_whale_liver,1.0,0.8277300898845916,0.9231764719150914,0.6237165331036026,0.5839001198830035,0.47109557041520245,0.2719093462662776,0.6825022436073613,0.6841162854945757,0.8065833989865004,0.6850213875576928,0.7899857007091465,0.719278598655406,0.5286961010921573,0.4489435056074344,0.5121332467029109,0.6298461707080121,0.5772035550483416
bowhead_whale_liver,0.8277300898845916,1.0,0.9297485675593679,0.729453278796176,0.6394810866079089,0.5388773215348185,0.3383741179558949,0.7730174087781075,0.7785332663796907,0.6790049910256678,0.7754762525813655,0.8736890632027058,0.8145719207842063,0.5591738180614569,0.5103960111913068,0.6131282893853465,0.693380496155032,0.6234410190264307
minke_liver,0.9231764719150914,0.9297485675593679,1.0,0.692347254017083,0.6455423627253681,0.5197703764542697,0.31801743778340913,0.7476002972933502,0.7516153560553112,0.6480641156386425,0.7275467681180834,0.856921042886526,0.7884509948912664,0.5209318578283816,0.466215175280821,0.5625110535696441,0.6499475735701372,0.5793552978069281
NMR_liver,0.6237165331036026,0.729453278796176,0.692347254017083,1.0,0.4965568906189072,0.46266543205444266,0.3263649770068802,0.5847860311413801,0.5961371269003599,0.6341200398602842,0.7205210166870238,0.8278918313897569,0.7242749929004192,0.5041062372727515,0.47371579813518483,0.5874948974878154,0.6425294842881665,0.5746602087792788
human_liver,0.5839001198830035,0.6394810866079089,0.6455423627253681,0.4965568906189072,1.0,0.728960820068277,0.427812964545153,0.7617933936885383,0.7446471727963583,0.4567391269738563,0.49478889119680536,0.5659417724675692,0.6955847159635357,0.5062677425656119,0.4167723293316779,0.5119919649522698,0.5271933392336179,0.4814844236868365
mouse_totalRNA_liver,0.47109557041520245,0.5388773215348185,0.5197703764542697,0.46266543205444266,0.728960820068277,1.0,0.6446251954067806,0.6652310614735593,0.6754636393786392,0.42043287611000485,0.5655711146300061,0.5077328027853565,0.6085229484106558,0.5516751446980661,0.6939430715721151,0.7490223913791028,0.5550998282188593,0.5349212758890417
mouse_mRNA_liver,0.2719093462662776,0.3383741179558949,0.31801743778340913,0.3263649770068802,0.427812964545153,0.6446251954067806,1.0,0.34816006646351355,0.3569456319795662,0.24959301534096084,0.32028006990835145,0.3348826213267037,0.3646637772213359,0.2863790933146166,0.3502233389494442,0.4310864573894164,0.29771337376173673,0.2709927831433547
cow_totalRNA_liver,0.6825022436073613,0.7730174087781075,0.7476002972933502,0.5847860311413801,0.7617933936885383,0.6652310614735593,0.34816006646351355,1.0,0.987646598795349,0.5719909064494857,0.6462966459553575,0.6874795127271022,0.8203228016352635,0.5839525158636899,0.4891978269797072,0.5234095847839716,0.7850296352020211,0.7435058880802083
cow_mRNA_liver,0.6841162854945757,0.7785332663796907,0.7516153560553112,0.5961371269003599,0.7446471727963583,0.6754636393786392,0.3569456319795662,0.987646598795349,1.0,0.575727809338449,0.6469930933609092,0.6982424714782212,0.8131421474659105,0.5655332839025662,0.4784898211331575,0.5260494532852728,0.765255936390077,0.7226842650678631
gray_whale_kidney,0.8065833989865004,0.6790049910256678,0.6480641156386425,0.6341200398602842,0.4567391269738563,0.42043287611000485,0.24959301534096084,0.5719909064494857,0.575727809338449,1.0,0.7081672512669088,0.7710701279362062,0.6761719951089161,0.5670383084366521,0.48647219124665636,0.551505370215726,0.6665799816923631,0.6260569256462964


In [3]:
val statGo = components.describe(features:_*)
val statComponents = components.describe(features:_*)
val statFunctions = functions.describe(features:_*)
val statProcesses = processes.describe(features:_*)

statGo = [summary: string, gray_whale_liver: string ... 17 more fields]
statComponents = [summary: string, gray_whale_liver: string ... 17 more fields]
statFunctions = [summary: string, gray_whale_liver: string ... 17 more fields]
statProcesses = [summary: string, gray_whale_liver: string ... 17 more fields]


[summary: string, gray_whale_liver: string ... 17 more fields]

In [5]:
write(statGo, az(statsPath + "/go_stats.tsv"))
write(statComponents, az(statsPath + "/components_stats.tsv"))
write(statFunctions, az(statsPath + "/functions_stats.tsv"))
write(statProcesses, az(statsPath + "/processes_stats.tsv"))

wasbs://storage@pipelines1.blob.core.windows.net/expressions/go/stats/processes_stats.tsv

In [6]:
%%dataframe statGo

summary,gray_whale_liver,bowhead_whale_liver,minke_liver,NMR_liver,human_liver,mouse_totalRNA_liver,mouse_mRNA_liver,cow_totalRNA_liver,cow_mRNA_liver,gray_whale_kidney,bowhead_whale_kidney,minke_kidney,NMR_kidney,human_kidney,mouse_totalRNA_kidney,mouse_mRNA_kidney,cow_totalRNA_kidney,cow_mRNA_kidney
count,1453,1243,1205,1017,1617,1582,1582,1437,1437,1453,1195,1205,1017,1617,1582,1582,1437,1437
mean,868.5354391699926,468.7891043725651,1139.1835879551854,3011.4510380943952,255.64292075386538,1939.6801433242754,595.5306853274335,2140.863913127349,2270.3778592651283,1192.0169739649018,188.7824074962342,1611.9041302257253,2234.358287944933,176.73478424180556,1616.8993346485413,507.8351420663703,2066.537954194845,2054.3890774996503
stddev,4899.940679867033,2553.2165129144614,6212.724892591675,14760.924799645785,1777.820935124624,13271.063811067626,5067.003835882542,14985.194700965862,15198.61551838991,6674.09065749058,1186.617703523776,8857.436828992848,12346.228517487874,1361.1948859049087,11896.801473627173,3168.1352716157044,13927.841305004195,14757.667618630821
min,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,82627.97828900004,43406.81879601012,103879.82613499998,189153.98861699994,34179.85539700001,238106.703232,116765.71726399995,407115.5446490002,402875.4094160001,107835.54761100012,24343.312661999982,139226.62839100015,251177.67484400002,30508.606157000006,311226.213938,71787.03052000001,315467.63018200017,373243.68564099993


In [52]:
%%dataframe components.describe

An error occurred converting DataFrame to html.
<console>:128: error: missing argument list for method describe in class Dataset
Unapplied methods are only converted to functions when a function type is expected.
You can make this conversion explicit by writing `describe _` or `describe(_)` instead of `describe`.
       components.describe
                  ^


In [48]:
val comp_count = components.count
val proc_count = processes.count
val fun_count = functions.count
val all_count = goTable.count

comp_count = 1739
proc_count = 12369
fun_count = 4396
all_count = 18597


18597

In [36]:
import org.apache.spark.sql.expressions._
def applyFunction2(df: DataFrame, uf: UserDefinedFunction, fields: List[String]): DataFrame = fields match {
    case Nil => df
    case head::tail => applyFunction( df.withColumn("z_score_"+head, uf(col(head))), uf, tail)
}

applyFunction2: (df: org.apache.spark.sql.DataFrame, uf: org.apache.spark.sql.expressions.UserDefinedFunction, fields: List[String])org.apache.spark.sql.DataFrame


In [43]:
import org.apache.spark.sql.functions._
import spark.implicits._
def fisher(r: Double): Double = 0.5 *  Math.log( (1 + r) / (1 - r) )
def zScore(r: Double, n: Double) = Math.sqrt((n-3)/ 1.06) * fisher(r)
val fisherize = udf[Double, Double]( fisher )
def scoreFunction2(n: Double):UserDefinedFunction = udf[Double, Double](r=>Math.sqrt((n-3)/ 1.06) * 0.5 *  Math.log( (1 + r) / (1 - r) ))

fisherize = UserDefinedFunction(<function1>,DoubleType,Some(List(DoubleType)))


lastException: Throwable = null
fisher: (r: Double)Double
zScore: (r: Double, n: Double)Double
scoreFunction2: (n: Double)org.apache.spark.sql.expressions.UserDefinedFunction


UserDefinedFunction(<function1>,DoubleType,Some(List(DoubleType)))

In [44]:
val zGo2 = applyFunction2(s, scoreFunction2(all_count), features.toList)
val zCorComponents2 = applyFunction2(sCorComponents, scoreFunction2(comp_count), features.toList)
val zCorFunctions2 = applyFunction2(sCorFunctions, scoreFunction2(fun_count), features.toList)
val zCorProcesses2 = applyFunction2(sCorProcesses, scoreFunction2(proc_count), features.toList)

zGo2 = [feature: string, gray_whale_liver: double ... 35 more fields]
zCorComponents2 = [feature: string, gray_whale_liver: double ... 35 more fields]
zCorFunctions2 = [feature: string, gray_whale_liver: double ... 35 more fields]
zCorProcesses2 = [feature: string, gray_whale_liver: double ... 35 more fields]


[feature: string, gray_whale_liver: double ... 35 more fields]

In [48]:
write(zGo2, az(statsPath + "/z_score_go.tsv"))
write(zCorComponents2, az(statsPath + "/z_score_components.tsv"))
write(zCorFunctions2, az(statsPath + "/z_score_functions.tsv"))
write(zCorProcesses2, az(statsPath + "/z_score_processes.tsv"))

lastException: Throwable = null


wasbs://storage@pipelines1.blob.core.windows.net/expressions/go/stats/z_score_processes.tsv

Uniref90 correlations for nonmodel animals
------------------------------------------------

In [1]:
val unirefPath = expressionsPath + "/" + "uniref90"

unirefPath = expressions/uniref90


expressions/uniref90

In [2]:
val bowheadAlaska_uniref90 = readTSV(az(unirefPath + "/bowhead_alaska_uniref90_all.tab"), true)
val bowheadGreenland_uniref90 = readTSV(az(unirefPath + "/bowhead_greenland_uniref90_all.tab"), true)
val minky_uniref90 = readTSV(az(unirefPath + "/minky_whale_uniref90_all.tab"), true)
val gray_whale_uniref90 = readTSV(az(unirefPath + "/gray_whale_uniref90_all.tab"), true)
val naked_mole_rat_uniref90 = readTSV(az(unirefPath + "/naked_mole_rat_uniref90_all.tab"), true)


bowheadAlaska_uniref90 = [uniref90: string, transcript: string ... 13 more fields]
bowheadGreenland_uniref90 = [uniref90: string, transcript: string ... 13 more fields]
minky_uniref90 = [uniref90: string, transcript: string ... 15 more fields]
gray_whale_uniref90 = [uniref90: string, transcript: string ... 15 more fields]
naked_mole_rat_uniref90 = [uniref90: string, transcript: string ... 15 more fields]


[uniref90: string, transcript: string ... 15 more fields]

In [39]:
def uselect(dataFrame: DataFrame, prefix: String) = {
    val liverNew = prefix + "_liver"
    val kidneyNew = prefix + "_kidney"
    dataFrame
        .withColumnRenamed("liver", liverNew)
        .withColumnRenamed("kidney", kidneyNew)
    .select("uniref90", liverNew, kidneyNew)
}

uselect: (dataFrame: org.apache.spark.sql.DataFrame, prefix: String)org.apache.spark.sql.DataFrame


In [66]:
val bowhead_outer = bowheadAlaska_uniref90
        .withColumnRenamed("liver", "bowhead_whale_liver")
        .select("uniref90", "bowhead_whale_liver")
    .join(bowheadGreenland_uniref90
      .withColumnRenamed("kidney", "bowhead_whale_kidney")
      .select("uniref90","bowhead_whale_kidney"), Seq("uniref90"), "outer")
    .na.fill(0)
val nonModelUniref90_outer = bowhead_outer
  .join(uselect(naked_mole_rat_uniref90, "NMR"), Seq("uniref90"), "outer")
  .join(uselect(minky_uniref90, "minke"), Seq("uniref90"), "outer")
  .join(uselect(gray_whale_uniref90, "gray_whale"), Seq("uniref90"), "outer").na.fill(0)
    .select("uniref90", "gray_whale_liver","bowhead_whale_liver", "minke_liver", "NMR_liver",
              "gray_whale_kidney","bowhead_whale_kidney", "minke_kidney", "NMR_kidney")

bowhead_outer = [uniref90: string, bowhead_whale_liver: double ... 1 more field]
nonModelUniref90_outer = [uniref90: string, gray_whale_liver: double ... 7 more fields]


lastException: Throwable = null


[uniref90: string, gray_whale_liver: double ... 7 more fields]

In [41]:
write(nonModelUniref90_outer, az(unirefPath + "/non_model_uniref90_outer.tsv"))

wasbs://storage@pipelines1.blob.core.windows.net/expressions/uniref90/nonModelUniref90_outer.tsv

In [60]:
val bowhead_inner = bowheadAlaska_uniref90
        .withColumnRenamed("liver", "bowhead_whale_liver")
        .select("uniref90", "bowhead_whale_liver")
    .join(bowheadGreenland_uniref90
      .withColumnRenamed("kidney", "bowhead_whale_kidney")
      .select("uniref90","bowhead_whale_kidney"), Seq("uniref90"), "inner")
    .na.fill(0)
val nonModelUniref90_inner = bowhead_inner
  .join(uselect(naked_mole_rat_uniref90, "NMR"), Seq("uniref90"), "inner")
  .join(uselect(minky_uniref90, "minke"), Seq("uniref90"), "inner")
  .join(uselect(gray_whale_uniref90, "gray_whale"), Seq("uniref90"), "inner").na.fill(0)
    .select("uniref90", "gray_whale_liver","bowhead_whale_liver", "minke_liver", "NMR_liver",
              "gray_whale_kidney","bowhead_whale_kidney", "minke_kidney", "NMR_kidney")

bowhead_inner = [uniref90: string, bowhead_whale_liver: double ... 1 more field]
nonModelUniref90_inner = [uniref90: string, gray_whale_liver: double ... 7 more fields]


lastException: Throwable = null


[uniref90: string, gray_whale_liver: double ... 7 more fields]

In [62]:
write(nonModelUniref90_inner, az(unirefPath + "/non_model_uniref90_inner.tsv"))

lastException: Throwable = null


wasbs://storage@pipelines1.blob.core.windows.net/expressions/uniref90/non_model_uniref90_inner.tsv

In [67]:
val pCorUniref90_inner = pearsonCorrellation(nonModelUniref90_inner, nonModelUniref90_inner.columns.tail)(spark)

pCorUniref90_inner = [feature: string, gray_whale_liver: double ... 7 more fields]


[feature: string, gray_whale_liver: double ... 7 more fields]

In [68]:
%%dataframe pCorUniref90_inner

feature,gray_whale_liver,bowhead_whale_liver,minke_liver,NMR_liver,gray_whale_kidney,bowhead_whale_kidney,minke_kidney,NMR_kidney
gray_whale_liver,1.0,0.8300586910031105,0.8469116693333069,0.6669585219776608,0.9298140542466256,0.8493871407603216,0.8462590061430213,0.5294741029129806
bowhead_whale_liver,0.8300586910031105,1.0,0.8000472869408696,0.45483848041601543,0.822258675210378,0.8771210752206287,0.8026170765946911,0.5211770501263002
minke_liver,0.8469116693333069,0.8000472869408696,1.0,0.5197339363588092,0.830027793793164,0.8102696110441489,0.9662833322999814,0.823327047028031
NMR_liver,0.6669585219776608,0.45483848041601543,0.5197339363588092,1.0,0.7334151283780421,0.49038781720551383,0.581252957671909,0.449795392863793
gray_whale_kidney,0.9298140542466256,0.822258675210378,0.830027793793164,0.7334151283780421,1.0,0.8362624204104355,0.8849154071710491,0.6248508934088939
bowhead_whale_kidney,0.8493871407603216,0.8771210752206287,0.8102696110441489,0.49038781720551383,0.8362624204104355,1.0,0.8374133453149177,0.5635818695611686
minke_kidney,0.8462590061430213,0.8026170765946911,0.9662833322999814,0.581252957671909,0.8849154071710491,0.8374133453149177,1.0,0.8251714962097114
NMR_kidney,0.5294741029129806,0.5211770501263002,0.823327047028031,0.449795392863793,0.6248508934088939,0.5635818695611686,0.8251714962097114,1.0


In [63]:
val sCorUniref90_inner = spearmanCorrellation(nonModelUniref90_inner, nonModelUniref90_inner.columns.tail)(spark)

sCorUniref90_inner = [feature: string, gray_whale_liver: double ... 7 more fields]


[feature: string, gray_whale_liver: double ... 7 more fields]

In [64]:
%%dataframe sCorUniref90_inner 

feature,gray_whale_liver,bowhead_whale_liver,minke_liver,NMR_liver,gray_whale_kidney,bowhead_whale_kidney,minke_kidney,NMR_kidney
gray_whale_liver,1.0,0.7639173336841389,0.8430350683951906,0.549761808756336,0.8347408304749393,0.5365711661878696,0.7430483190607209,0.700989262822804
bowhead_whale_liver,0.7639173336841389,1.0,0.8271636509527601,0.5877812506699471,0.7439277239332438,0.687662277965784,0.7265230142709078,0.709511650790017
minke_liver,0.8430350683951906,0.8271636509527601,1.0,0.6336283978593613,0.7990513668612917,0.6494176766396411,0.8539677496783272,0.7442344088031906
NMR_liver,0.549761808756336,0.5877812506699471,0.6336283978593613,1.0,0.6469130529791486,0.5990136932321215,0.6652075366481272,0.7261078254797796
gray_whale_kidney,0.8347408304749393,0.7439277239332438,0.7990513668612917,0.6469130529791486,1.0,0.7410226620018905,0.8658049826038999,0.6906618601830136
bowhead_whale_kidney,0.5365711661878696,0.687662277965784,0.6494176766396411,0.5990136932321215,0.7410226620018905,1.0,0.7279144448062425,0.5731992179358378
minke_kidney,0.7430483190607209,0.7265230142709078,0.8539677496783272,0.6652075366481272,0.8658049826038999,0.7279144448062425,1.0,0.6951876874316372
NMR_kidney,0.700989262822804,0.709511650790017,0.7442344088031906,0.7261078254797796,0.6906618601830136,0.5731992179358378,0.6951876874316372,1.0


In [69]:
%%dataframe p

feature,gray_whale_liver,bowhead_whale_liver,minke_liver,NMR_liver,human_liver,mouse_totalRNA_liver,mouse_mRNA_liver,cow_totalRNA_liver,cow_mRNA_liver,gray_whale_kidney,bowhead_whale_kidney,minke_kidney,NMR_kidney,human_kidney,mouse_totalRNA_kidney,mouse_mRNA_kidney,cow_totalRNA_kidney,cow_mRNA_kidney
gray_whale_liver,1.0,0.8277300898845916,0.9231764719150914,0.6237165331036026,0.5839001198830035,0.47109557041520245,0.2719093462662776,0.6825022436073613,0.6841162854945757,0.8065833989865004,0.6850213875576928,0.7899857007091465,0.719278598655406,0.5286961010921573,0.4489435056074344,0.5121332467029109,0.6298461707080121,0.5772035550483416
bowhead_whale_liver,0.8277300898845916,1.0,0.9297485675593679,0.729453278796176,0.6394810866079089,0.5388773215348185,0.3383741179558949,0.7730174087781075,0.7785332663796907,0.6790049910256678,0.7754762525813655,0.8736890632027058,0.8145719207842063,0.5591738180614569,0.5103960111913068,0.6131282893853465,0.693380496155032,0.6234410190264307
minke_liver,0.9231764719150914,0.9297485675593679,1.0,0.692347254017083,0.6455423627253681,0.5197703764542697,0.31801743778340913,0.7476002972933502,0.7516153560553112,0.6480641156386425,0.7275467681180834,0.856921042886526,0.7884509948912664,0.5209318578283816,0.466215175280821,0.5625110535696441,0.6499475735701372,0.5793552978069281
NMR_liver,0.6237165331036026,0.729453278796176,0.692347254017083,1.0,0.4965568906189072,0.46266543205444266,0.3263649770068802,0.5847860311413801,0.5961371269003599,0.6341200398602842,0.7205210166870238,0.8278918313897569,0.7242749929004192,0.5041062372727515,0.47371579813518483,0.5874948974878154,0.6425294842881665,0.5746602087792788
human_liver,0.5839001198830035,0.6394810866079089,0.6455423627253681,0.4965568906189072,1.0,0.728960820068277,0.427812964545153,0.7617933936885383,0.7446471727963583,0.4567391269738563,0.49478889119680536,0.5659417724675692,0.6955847159635357,0.5062677425656119,0.4167723293316779,0.5119919649522698,0.5271933392336179,0.4814844236868365
mouse_totalRNA_liver,0.47109557041520245,0.5388773215348185,0.5197703764542697,0.46266543205444266,0.728960820068277,1.0,0.6446251954067806,0.6652310614735593,0.6754636393786392,0.42043287611000485,0.5655711146300061,0.5077328027853565,0.6085229484106558,0.5516751446980661,0.6939430715721151,0.7490223913791028,0.5550998282188593,0.5349212758890417
mouse_mRNA_liver,0.2719093462662776,0.3383741179558949,0.31801743778340913,0.3263649770068802,0.427812964545153,0.6446251954067806,1.0,0.34816006646351355,0.3569456319795662,0.24959301534096084,0.32028006990835145,0.3348826213267037,0.3646637772213359,0.2863790933146166,0.3502233389494442,0.4310864573894164,0.29771337376173673,0.2709927831433547
cow_totalRNA_liver,0.6825022436073613,0.7730174087781075,0.7476002972933502,0.5847860311413801,0.7617933936885383,0.6652310614735593,0.34816006646351355,1.0,0.987646598795349,0.5719909064494857,0.6462966459553575,0.6874795127271022,0.8203228016352635,0.5839525158636899,0.4891978269797072,0.5234095847839716,0.7850296352020211,0.7435058880802083
cow_mRNA_liver,0.6841162854945757,0.7785332663796907,0.7516153560553112,0.5961371269003599,0.7446471727963583,0.6754636393786392,0.3569456319795662,0.987646598795349,1.0,0.575727809338449,0.6469930933609092,0.6982424714782212,0.8131421474659105,0.5655332839025662,0.4784898211331575,0.5260494532852728,0.765255936390077,0.7226842650678631
gray_whale_kidney,0.8065833989865004,0.6790049910256678,0.6480641156386425,0.6341200398602842,0.4567391269738563,0.42043287611000485,0.24959301534096084,0.5719909064494857,0.575727809338449,1.0,0.7081672512669088,0.7710701279362062,0.6761719951089161,0.5670383084366521,0.48647219124665636,0.551505370215726,0.6665799816923631,0.6260569256462964


Uniref computations for model organisms
---------------------------------------

In [90]:
val mouse_mRNA = readTSV(az(transcriptsPath + "/mouse_mRNA_transcripts_all.tsv"), true)
val mouse_totalRNA =  readTSV(az(transcriptsPath + "/mouse_totalRNA_transcripts_all.tsv"), true)
val cow_mRNA = readTSV(az(transcriptsPath + "/cow_mRNA_transcripts_all.tsv"), true)
val cow_totalRNA =  readTSV(az(transcriptsPath + "/cow_totalRNA_transcripts_all.tsv"), true)
val human_totalRNA = readTSV(az(transcriptsPath + "/human_transcripts_all.tsv"), true)

mouse_mRNA = [transcript: string, uniref90: string ... 13 more fields]
mouse_totalRNA = [transcript: string, uniref90: string ... 13 more fields]
cow_mRNA = [transcript: string, uniref90: string ... 13 more fields]
cow_totalRNA = [transcript: string, uniref90: string ... 13 more fields]
human_totalRNA = [transcript: string, uniref90: string ... 13 more fields]


[transcript: string, uniref90: string ... 13 more fields]

In [93]:
%%dataframe cow_totalRNA

transcript,uniref90,go,liver,kidney,avg_expression,uniprot_ac,taxon,uniprot_id,entrez,refSeq,gi,uniparc,pubmed,embl
ENSBTAT00000025946,UniRef90_F1MN15,GO:0004536; GO:0016888; GO:0006308,3.985021,11.832584,15.817605,F1MN15,9913,F1MN15_BOVIN,null,null,null,UPI0000F30D1B,19393038,DAAA02053751
ENSBTAT00000009797,UniRef90_Q9NZJ9,GO:0005737; GO:0005634; GO:0034431; GO:0034432; GO:0008486; GO:0000298; GO:0050072; GO:1901911; GO:1901909; GO:1901907; GO:0071543,7.028422,64.554945,71.58336700000001,A2VDV0,9913,A2VDV0_BOVIN,614183,NP_001075087.1,125991926; 124829074,UPI0000616276,19393038,DAAA02012773; BC133414
ENSBTAT00000008621,UniRef90_Q9R1P4,GO:0005737; GO:0005634; GO:0005839; GO:0019773; GO:0001530; GO:0004298; GO:0002376; GO:0002862; GO:0006511,56.054289,70.186761,126.24105,Q3T0X5,9913,PSA1_BOVIN,515503,NP_001030387.1,21465661; 74354782; 122140416; 78369458; 21465647,UPI00005B61ED,12015144,BC102216
ENSBTAT00000002123,UniRef90_F1N5T9,GO:0070062; GO:0005794; GO:0016021; GO:0043209; GO:0005886; GO:0015194; GO:0051607; GO:0009597; GO:0045087; GO:0042552; GO:0006658; GO:0006665,14.145071,15.494706,29.639777000000002,F1N5T9,9913,F1N5T9_BOVIN,null,null,null,UPI0002264BBB,19393038,DAAA02027840
ENSBTAT00000000236,UniRef90_F1MVX6,GO:0000786; GO:0005634; GO:0003677; GO:0046982; GO:0006334,1.862637,1.867203,3.7298400000000003,F1MVX6,9913,F1MVX6_BOVIN,null,null,null,UPI00022650F9,19393038,DAAA02032276
ENSBTAT00000064367,UniRef90_W5PQ94,GO:0016021; GO:0005886; GO:0004930; GO:0004984,0.081154,0.053759,0.134913,G3N2Y6,9913,G3N2Y6_BOVIN,null,null,null,UPI000226505E,19393038,DAAA02028046
ENSBTAT00000003860,UniRef90_F1MDR9,GO:0004722,34.219832,30.02334,64.243172,F1MDR9,9913,F1MDR9_BOVIN,null,null,null,UPI0002264D79,19393038,DAAA02054383; DAAA02054384
ENSBTAT00000025094,UniRef90_P12081,GO:0005829; GO:0005739; GO:0005524; GO:0004821; GO:0042802; GO:0016874; GO:0006427; GO:0032543,28.883329,41.151501,70.03483,Q2KI84,9913,SYHC_BOVIN,510937,NP_001039531.1,114051578; 86826556; 110287977,UPI0000695E6F,null,BC112731
ENSBTAT00000054839,UniRef90_F1MNX4,null,0.017913,0.305231,0.323144,F1MNX4,9913,F1MNX4_BOVIN,null,null,null,UPI0002263F4B,19393038,DAAA02058454
ENSBTAT00000022373,UniRef90_Q9NP64,GO:0003676; GO:0008270,31.627287,36.404036,68.031323,Q3T0H4,9913,Q3T0H4_BOVIN,538816,NP_001029824.1; XP_005203034.1; XP_005203035.1,95769249; 528940528; 77736247; 528940530; 74354076,UPI00005B6221,11282978; 16305752; 19393038,DAAA02006258; DAAA02006259; DAAA02006260; DAAA02006261; DAAA02006262; DAAA02006263; BC102395; BT025465


In [87]:
%%dataframe mouse_mapping

uniprot_ac,uniprot_id,entrez,refSeq,gi,pdb,go,uniref100,uniref90,uniref50,uniparc,pir,taxon,mim,unigene,pubmed,embl,embl_cds,ensembl,ensembl_trs,ensembl_pro,additional_pubmed
Q9CQV8,1433B_MOUSE,54401,NP_061223.2; XP_006499972.1,1009083986; 409974018; 74219534; 31543974; 3065925; 568917832; 18202836; 74191394,4GNT:A; 5F74:A,GO:0005737; GO:0030659; GO:0005829; GO:0042470; GO:0005634; GO:0048471; GO:0043234; GO:0017053; GO:0045296; GO:0019899; GO:0042826; GO:0042802; GO:0051219; GO:0050815; GO:0008022; GO:0032403; GO:0019904; GO:0003714; GO:0051220; GO:0045744; GO:0035308; GO:0045892; GO:0043085; GO:0051291; GO:0006605,UniRef100_Q9CQV8,UniRef90_P35213,UniRef50_P27348,UPI0000004168,null,10090,null,Mm.34319; Mm.491117,16141072; 9705322; 15883195; 16800626; 18604201; 21183079; 20478393; 26047703,AF058797; AK002632; AK004872; AK011389; AK083367; AK144061; AK150414; AK151294; AK158932,AAC14343.1; BAB22246.1; BAB23631.1; BAB27587.1; BAC38886.1; BAE25678.1; BAE29538.1; BAE30278.1; BAE34730.1,ENSMUSG00000018326,ENSMUST00000018470,ENSMUSP00000018470,11217851; 12008017; 12446771; 12466851; 12937170; 14610273; 14679215; 15226823; 15240115; 16099986; 16282323; 16602821; 16635246; 16959763; 17923476; 18237277; 18239087; 18332103; 18606808; 18799693; 19014373; 19429675; 19841136; 19915011; 19955570; 20598904; 21267068; 21390248; 23071090; 23177959; 23615913; 23791194; 24026679; 24206222; 24726955; 26214742; 26984404; 11875115; 12796778; 16613846; 18628823; 18687683; 19223768; 19423573; 19834914; 20410134; 20412061; 20697350; 21112954; 26045162; 26514267; 9422753
P62259,1433E_MOUSE,22627,NP_033562.3,60391192; 226874906,null,GO:0030424; GO:0090724; GO:0005737; GO:0030659; GO:0005829; GO:0005871; GO:0042470; GO:0005739; GO:0005634; GO:0005886; GO:0045296; GO:0005246; GO:0019899; GO:0042826; GO:0042802; GO:0044325; GO:0051219; GO:0050815; GO:0015459; GO:0032403; GO:0019904; GO:0046982; GO:0097110; GO:0031625; GO:0034605; GO:0021987; GO:0021766; GO:0000165; GO:1905913; GO:1901020; GO:1902309; GO:0035308; GO:0001764; GO:0046827; GO:0006605; GO:0051480; GO:0060306; GO:1901016,UniRef100_P62258,UniRef90_P62258,UniRef50_P62258,UPI0000021A46,I48337,10090,null,Mm.234700; Mm.471625,7750640; 11471062; 15489334; 10409742; 11278287; 11533041; 12796778; 15722337; 17320046; 21183079; 21078877; 26047703,Z19599; D87663; AF483478; AF483479; BC058686,CAA79659.1; BAA13424.1; AAL90752.1; AAL90753.1; AAH58686.1,ENSMUSG00000020849,ENSMUST00000067664,ENSMUSP00000070993,11784696; 14966136; 15364926; 17720156; 18687683; 21152247; 21245341; 21287435; 21458426; 22065591; 23071090; 23326474; 25059436; 26208633; 27001213; 27845227; 10441394; 10725249; 11217851; 11585834; 12446771; 12466851; 12520002; 12904583; 12905154; 14517258; 14610273; 14613977; 14651853; 14681479; 15166223; 15284227; 15741177; 15979417; 16141072; 16482218; 16602821; 16615898; 16956611; 16959763; 18059471; 18331715; 18332103; 18658164; 18799693; 19014373; 19799774; 19865483; 20410134; 21092859; 21145499; 21242966; 21267068; 21346153; 21390248; 21423666; 21436042; 21677750; 22982415; 23055941; 23177959; 23615913; 23673330; 23831578; 24721909; 25186760; 27580238; 7671812; 8684458; 8798343; 8858348; 9182797; 9199935; 9811942; 16216880; 16635246; 18614564; 19423573; 19455133; 20564261; 20697350; 21274006; 21565611; 24390569; 24687852; 9144171; 9452471
P68510,1433F_MOUSE,22629,NP_035868.1,74226056; 17976969; 1526541; 74139366; 74140682; 3335652; 55976655; 1354808; 26346472; 74214318; 148705461; 38181914; 14198255; 6756037,null,GO:0005737; GO:0030659; GO:0005829; GO:0014704; GO:0005886; GO:0003779; GO:0019899; GO:0035259; GO:0042802; GO:0044325; GO:0019904; GO:0046982; GO:0017080; GO:0007010; GO:0006713; GO:0042921; GO:0006886; GO:0086010; GO:0043066; GO:0050774; GO:0045893; GO:0007088; GO:2000649; GO:0002028,UniRef100_P68510,UniRef90_P68510,UniRef50_P62258,UPI000000109B,null,10090,null,Mm.332314,9197417; 9738002; 12008017; 16141072; 15489334; 9705322; 11533041; 18397886; 21183079; 20478393; 26047703,U57311; D8

Validational data
=========

In [24]:
val validationPath = expressionsPath + "/" + "validation"
val mouseValidationPath = validationPath + "/" + "mouse"
val jenageValidationPath = mouseValidationPath + "/" + "GSE75192"
val mouseColsValidationPath = jenageValidationPath + "/" + "expressions_columns_GSE75192.tsv"
val mouseRowsValidationPath = jenageValidationPath + "/" + "expressions_rows_GSE75192.tsv"


validationPath = expressions/validation
mouseValidationPath = expressions/validation/mouse
jenageValidationPath = expressions/validation/mouse/GSE75192
mouseColsValidationPath = expressions/validation/mouse/GSE75192/expressions_columns_GSE75192.tsv
mouseRowsValidationPath = expressions/validation/mouse/GSE75192/expressions_rows_GSE75192.tsv


expressions/validation/mouse/GSE75192/expressions_rows_GSE75192.tsv

In [30]:
val mouseRows = readTSV(az(mouseRowsValidationPath), true)

Name: java.lang.OutOfMemoryError
Message: Java heap space
StackTrace:   at java.util.Arrays.copyOf(Arrays.java:3332)
  at java.lang.AbstractStringBuilder.ensureCapacityInternal(AbstractStringBuilder.java:124)
  at java.lang.AbstractStringBuilder.append(AbstractStringBuilder.java:448)
  at java.lang.StringBuilder.append(StringBuilder.java:136)
  at scala.collection.mutable.StringBuilder.append(StringBuilder.scala:200)
  at scala.collection.TraversableOnce$$anonfun$addString$1.apply(TraversableOnce.scala:364)
  at scala.collection.immutable.List.foreach(List.scala:381)
  at scala.collection.TraversableOnce$class.addString(TraversableOnce.scala:357)
  at scala.collection.AbstractTraversable.addString(Traversable.scala:104)
  at scala.collection.TraversableOnce$class.mkString(TraversableOnce.scala:323)
  at scala.collection.AbstractTraversable.mkString(Traversable.scala:104)
  at scala.collection.TraversableOnce$class.mkString(TraversableOnce.scala:325)
  at scala.collection.AbstractTraver

In [48]:
%%dataframe mouseRows

An error occurred converting DataFrame to html.
lastException: Throwable = null
<console>:138: error: not found: value mouseRows
       mouseRows
       ^


In [28]:
val mouseCols = readTSV(az(mouseColsValidationPath), true)

mouseCols = [transcripts\samples: string, GSM1944768: double ... 62 more fields]


[transcripts\samples: string, GSM1944768: double ... 62 more fields]

In [32]:
%%dataframe mouseCols

transcripts\samples,GSM1944768,GSM1944769,GSM1944770,GSM1944771,GSM1944772,GSM1944773,GSM1944774,GSM1944775,GSM1944776,GSM1944777,GSM1944778,GSM1944779,GSM1944780,GSM1944781,GSM1944782,GSM1944783,GSM1944784,GSM1944785,GSM1944786,GSM1944787,GSM1944788,GSM1944789,GSM1944790,GSM1944791,GSM1944792,GSM1944793,GSM1944794,GSM1944795,GSM1944796,GSM1944797,GSM1944798,GSM1944799,GSM1944800,GSM1944801,GSM1944802,GSM1944803,GSM1944804,GSM1944805,GSM1944806,GSM1944807,GSM1944808,GSM1944809,GSM1944810,GSM1944811,GSM1944812,GSM1944813,GSM1944814,GSM1944815,GSM1944816,GSM1944817,GSM1944818,GSM1944819,GSM1944820,GSM1944821,GSM1944822,GSM1944823,GSM1944824,GSM1944825,GSM1944826,GSM1944827,GSM1944828,GSM1944829,GSM1944830
ENSMUST00000193812.1|ENSMUSG00000102693.1|OTTMUSG00000049935.1|OTTMUST00000127109.1|RP23-271O17.1-001|RP23-271O17.1|1070|TEC|,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.024182,0.020793,0.016223,0.04766,0.020777,0.0,0.0,0.0,0.0,0.012129,0.0,0.0,0.0,0.0,0.020563,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.036685,0.018082,0.0,0.033083,0.0,0.0,0.028305,0.0,0.0,0.044858,0.0,0.025776
ENSMUST00000082908.1|ENSMUSG00000064842.1|-|-|Gm26206-201|Gm26206|110|snRNA|,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSMUST00000162897.1|ENSMUSG00000051951.5|OTTMUSG00000026353.2|OTTMUST00000086625.1|AC157543.1-003|Xkr4|4153|processed_transcript|,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.845847,3.929305,3.280159,1.721415,2.849152,3.914385,1.89188,3.050555,1.194886,3.317348,1.350467,3.52716,2.531776,3.370729,1.981201,2.595443,2.267062,2.632438,3.331712,3.65756,2.411469,3.501615,1.989149,3.019143,3.503243,2.980232,3.297474,2.425381,2.472874,2.920016,2.310033,2.314357,2.613919,1.601334,2.743659,3.481774,3.06037,1.239043,1.907532
ENSMUST00000159265.1|ENSMUSG00000051951.5|OTTMUSG00000026353.2|OTTMUST00000086624.1|AC157543.1-002|Xkr4|2989|processed_transcript|,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.17336,3.139969,3.210255,1.736156,2.076496,2.509397,1.376288,2.719016,1.028211,2.835558,1.046355,2.888735,1.800355,2.632922,1.491177,2.038503,1.650463,2.307781,2.699214,2.13219,1.898486,2.44241,1.715781,2.43412,2.697341,2.632112,2.703276,1.916514,1.874155,2.469174,1.697529,1.879833,2.243523,1.540279,2.356135,2.51088,2.536061,0.882864,1.558731
ENSMUST00000070533.4|ENSMUSG00000051951.5|OTTMUSG00000026353.2|OTTMUST00000065166.1|AC157543.1-001|Xkr4|3634|protein_coding|,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.295981,3.137039,3.119776,2.456489,2.184665,3.310844,1.521686,3.3146,1.426211,3.055218,1.190453,2.82577,2.126554,3.034597,1.687767,2.623207,1.834155,2.276686,2.652952,2.755665,2.280889,3.132647,1.598027,2.482539,2.908216,2.667507,3.47796,2.148933,2.6256,3.088543,2.083587,2.102174,2.530062,1.64131,2.946637,3.087038,2.482002,1.119524,1.773834
ENSMUST00000192857.1|ENSMUSG00000102851.1|OTTMUSG00000049958.1|OTTMUST00000127143.1|RP23-317L18.1-001|RP23-317L18.1|480|processed_pseudogene|,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.11532,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.055959,0.0,0.106961,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.058736,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSMUST00000195335.1|ENSMUSG00000103377.1|OTTMUSG00000049960.1|OTTMUST00000127145.1|RP23-317L18.4-001|RP23-317L18.4|2819|TEC|,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.021717,0.006544,0.006643,0.0,0.005075,0.013274,0.007396,0.011441,0.005954,0.020022,0.015499,0.00525,0.01363,0.012002,0.01096,0.006569,0.0,0.009893,0.0,0.0,0.01006,0.01

In [29]:
mouseCols.count

133618

In [40]:
def span(from: String, to: String, prefix: String) = {
    val f = from.replace(prefix, "").toLong
    val t = to.replace(prefix, "").toLong
    for(v <- f to t) yield prefix + v
}

span: (from: String, to: String, prefix: String)scala.collection.immutable.IndexedSeq[String]


In [55]:
val renamings = (span("GSM1944768", "GSM1944791", "GSM").map(v=>v->s"blood_${v}") ++
span("GSM1944792",  "GSM1944830", "GSM").map(v=>v->s"brain_${v}") ++
span("GSM1944831", "GSM1944854", "GSM").map(v=>v->s"liver_${v}") ++
span("GSM1944855", "GSM1944879", "GSM").map(v=>v->s"skin_${v}") ++ Seq(
"GSM2795210" -> "blood_GSM2795210", "GSM2795211" -> "brain_GSM2795211", "GSM2795212"-> "liver_GSM2795212"
)).toMap

renamings = Map(GSM1944837 -> liver_GSM1944837, GSM1944781 -> blood_GSM1944781, GSM1944842 -> liver_GSM1944842, GSM1944858 -> skin_GSM1944858, GSM1944770 -> blood_GSM1944770, GSM1944791 -> blood_GSM1944791, GSM1944832 -> liver_GSM1944832, GSM1944861 -> skin_GSM1944861, GSM1944785 -> blood_GSM1944785, GSM1944853 -> liver_GSM1944853, GSM1944794 -> brain_GSM1944794, GSM1944864 -> skin_GSM1944864, GSM1944809 -> brain_GSM1944809, GSM1944774 -> blood_GSM1944774, GSM1944871 -> skin_GSM1944871, GSM1944847 -> liver_GSM1944847, GSM1944805 -> brain_GSM1944805, GSM1944816 -> brain_GSM1944816, GSM1944798 -> brain_GSM1944798, GSM1944808 -> brain_GSM1944808, GSM1944778 -> blood_GSM1944778, GSM1944860 -> skin_GSM1944860, GSM1944786 -> blood_GSM1944786, GSM...


Map(GSM1944837 -> liver_GSM1944837, GSM1944781 -> blood_GSM1944781, GSM1944842 -> liver_GSM1944842, GSM1944858 -> skin_GSM1944858, GSM1944770 -> blood_GSM1944770, GSM1944791 -> blood_GSM1944791, GSM1944832 -> liver_GSM1944832, GSM1944861 -> skin_GSM1944861, GSM1944785 -> blood_GSM1944785, GSM1944853 -> liver_GSM1944853, GSM1944794 -> brain_GSM1944794, GSM1944864 -> skin_GSM1944864, GSM1944809 -> brain_GSM1944809, GSM1944774 -> blood_GSM1944774, GSM1944871 -> skin_GSM1944871, GSM1944847 -> liver_GSM1944847, GSM1944805 -> brain_GSM1944805, GSM1944816 -> brain_GSM1944816, GSM1944798 -> brain_GSM1944798, GSM1944808 -> brain_GSM1944808, GSM1944778 -> blood_GSM1944778, GSM1944860 -> skin_GSM1944860, GSM1944786 -> blood_GSM1944786, GSM1944797 -> brain_GSM1944797, GSM1944789 -> blood_GSM1944789, GSM1944875 -> skin_GSM1944875, GSM1944869 -> skin_GSM1944869, GSM1944820 -> brain_GSM1944820, GSM1944827 -> brain_GSM1944827, GSM1944819 -> brain_GSM1944819, GSM1944769 -> blood_GSM1944769, GSM1944848 

In [62]:
def rename(dataFrame: DataFrame, renamings: Map[String, String]) =   {
      val newColumns = dataFrame.columns.map(c=> renamings.getOrElse(c, c))
      dataFrame.toDF(newColumns:_*)
    }
def renameFunction(renamings: Map[String, String]) = udf[String, String](c=> renamings.getOrElse(c, c))

rename: (dataFrame: org.apache.spark.sql.DataFrame, renamings: Map[String,String])org.apache.spark.sql.DataFrame
renameFunction: (renamings: Map[String,String])org.apache.spark.sql.expressions.UserDefinedFunction


In [149]:
val mouseCorPearsonComponents = pearsonCorrellation(mouseCols, mouseCols.columns.tail)(spark)
val mouseCorSpearmanComponents = spearmanCorrellation(mouseCols, mouseCols.columns.tail)(spark)


mouseCorPearsonComponents = [feature: string, GSM1944768: double ... 62 more fields]
mouseCorSpearmanComponents = [feature: string, GSM1944768: double ... 62 more fields]


[feature: string, GSM1944768: double ... 62 more fields]

In [151]:
val mouseCorSpearmanComponents2 = rename(mouseCorSpearmanComponents, renamings)
.withColumn("feature", renameFunction(renamings)($"feature"))

val mouseCorPearsonComponents2  = rename(mouseCorPearsonComponents, renamings)
.withColumn("feature", renameFunction(renamings)($"feature"))

mouseCorSpearmanComponents2 = [feature: string, blood_GSM1944768: double ... 62 more fields]
mouseCorPearsonComponents2 = [feature: string, blood_GSM1944768: double ... 62 more fields]


[feature: string, blood_GSM1944768: double ... 62 more fields]

In [67]:
%%dataframe mouseCorPearsonComponents2

feature,blood_GSM1944768,blood_GSM1944769,blood_GSM1944770,blood_GSM1944771,blood_GSM1944772,blood_GSM1944773,blood_GSM1944774,blood_GSM1944775,blood_GSM1944776,blood_GSM1944777,blood_GSM1944778,blood_GSM1944779,blood_GSM1944780,blood_GSM1944781,blood_GSM1944782,blood_GSM1944783,blood_GSM1944784,blood_GSM1944785,blood_GSM1944786,blood_GSM1944787,blood_GSM1944788,blood_GSM1944789,blood_GSM1944790,blood_GSM1944791,brain_GSM1944792,brain_GSM1944793,brain_GSM1944794,brain_GSM1944795,brain_GSM1944796,brain_GSM1944797,brain_GSM1944798,brain_GSM1944799,brain_GSM1944800,brain_GSM1944801,brain_GSM1944802,brain_GSM1944803,brain_GSM1944804,brain_GSM1944805,brain_GSM1944806,brain_GSM1944807,brain_GSM1944808,brain_GSM1944809,brain_GSM1944810,brain_GSM1944811,brain_GSM1944812,brain_GSM1944813,brain_GSM1944814,brain_GSM1944815,brain_GSM1944816,brain_GSM1944817,brain_GSM1944818,brain_GSM1944819,brain_GSM1944820,brain_GSM1944821,brain_GSM1944822,brain_GSM1944823,brain_GSM1944824,brain_GSM1944825,brain_GSM1944826,brain_GSM1944827,brain_GSM1944828,brain_GSM1944829,brain_GSM1944830
blood_GSM1944768,1.0,0.9800146988978149,0.9980087845450593,0.9758984471327348,0.986526585240169,0.9276551607393562,0.9791040387800648,0.929036324665687,0.9449397845871823,0.9641924361531625,0.9784081617681026,0.9867468065456799,0.9623295917802204,0.9378967153445574,0.9784607518723251,0.8918602100200701,0.9715442969378143,0.9396622299097261,0.8863357660557402,0.9725706852637802,0.988507774401264,0.981665267940832,0.9214699330142266,0.9523355736075579,0.13322074282954538,0.13007749507677693,0.12430065903426131,0.11984469119837797,0.1264362620183099,0.12621668422793292,0.12327497511088473,0.1361992892498205,0.12472971847747451,0.1324864736809431,0.1163458408510198,0.1302367485132671,0.12570676293678015,0.13103332769382964,0.1257911301490447,0.13258728776059478,0.13256833166997564,0.13130508398730964,0.12924171253429742,0.12581680674504703,0.12742569028180592,0.1276917885374066,0.12074101230060709,0.13097112738176522,0.127643776228311,0.13089048286712762,0.13123073451086795,0.13096110397492675,0.1336816865493354,0.12979444187040362,0.12982095936395782,0.14167960299828475,0.13406930284499433,0.12990369921136505,0.13537989594576788,0.1326771857191741,0.13295498965883748,0.13510561974411164,0.13547950554513538
blood_GSM1944769,0.9800146988978149,1.0,0.9873082932238477,0.9792266818201666,0.9925400706363682,0.9463849974618574,0.9911875507224625,0.9497565957713517,0.9703954306507344,0.9711156481179406,0.9862687774484886,0.9720372928855354,0.9778495507725905,0.9720888568525774,0.9833562251738096,0.9568898326685787,0.985907173742579,0.9611640672403833,0.8954908193872523,0.9610317584428727,0.9670575334692151,0.9839672724382208,0.9720806889481292,0.9666737743402176,0.1963085781860788,0.19137432747408686,0.18347657791011307,0.17695269775875713,0.1876935374653852,0.18701853695495402,0.1854327465468714,0.20102661719679948,0.18826718924257252,0.19565359052612893,0.17625420116279628,0.1917541782192559,0.18710407174711796,0.19244873474992608,0.18786749559176869,0.19635573335767062,0.19770064884204125,0.19498489239464628,0.1909936955715739,0.18584614700192223,0.18899326164542626,0.18938690283108026,0.1817019459281103,0.19487331991207132,0.18925663537455406,0.19366792497007615,0.19237743466613597,0.19347878464221796,0.19633475079272156,0.18985153170351055,0.19303599571029462,0.20882340130223861,0.19753058367093174,0.19419377958408438,0.1993077795332422,0.19612485547553588,0.19590444128991855,0.20198449429566204,0.201137787641386
blood_GSM1944770,0.9980087845450593,0.9873082932238477,1.0,0.9852364109878083,0.9935249700926262,0.9419413464368868,0.9874457788767739,0.9458455142830883,0.9602972252203994,0.9754040297014612,0.9859532556152789,0.9908906342793611,0.9752277615634076,0.9547968792483008,0.9859215648390596,0.9125072649181599,0.9745870399338278,0.9545059226710972,0.9017265175435737,0.9797414165351054,0.9836228155138017,0.9825437564303823,0.9387566439649436,0.965009592717647,0.13886559701

In [69]:
write(mouseCorSpearmanComponents2, az(corPath + "/spearman_transcripts_mouse_GSE75192.tsv"))
write(mouseCorPearsonComponents2, az(corPath + "/pearson_transcripts_mouse_GSE75192.tsv"))

wasbs://storage@pipelines1.blob.core.windows.net/expressions/go/grouped/correlations/pearson_transcripts_mouse_GSE75192.tsv

Mice GO correlations
------------------------

In [105]:
val mc =  mouseCols
.withColumnRenamed("transcripts\\samples","transcript")
.withColumn("transcript", simplifyFunction($"transcript"))

mc = [transcript: string, GSM1944768: double ... 62 more fields]


[transcript: string, GSM1944768: double ... 62 more fields]

In [108]:
val miceWithGO = mc.join(mouse_mapping
        .withColumnRenamed("ensembl_trs","transcript")
        .select("go", "transcript"),
        Seq("transcript"), "outer") 

miceWithGO = [transcript: string, GSM1944768: double ... 63 more fields]


[transcript: string, GSM1944768: double ... 63 more fields]

In [109]:
%%dataframe miceWithGO

transcript,GSM1944768,GSM1944769,GSM1944770,GSM1944771,GSM1944772,GSM1944773,GSM1944774,GSM1944775,GSM1944776,GSM1944777,GSM1944778,GSM1944779,GSM1944780,GSM1944781,GSM1944782,GSM1944783,GSM1944784,GSM1944785,GSM1944786,GSM1944787,GSM1944788,GSM1944789,GSM1944790,GSM1944791,GSM1944792,GSM1944793,GSM1944794,GSM1944795,GSM1944796,GSM1944797,GSM1944798,GSM1944799,GSM1944800,GSM1944801,GSM1944802,GSM1944803,GSM1944804,GSM1944805,GSM1944806,GSM1944807,GSM1944808,GSM1944809,GSM1944810,GSM1944811,GSM1944812,GSM1944813,GSM1944814,GSM1944815,GSM1944816,GSM1944817,GSM1944818,GSM1944819,GSM1944820,GSM1944821,GSM1944822,GSM1944823,GSM1944824,GSM1944825,GSM1944826,GSM1944827,GSM1944828,GSM1944829,GSM1944830,go
ENSMUST00000001780,1.457035,2.377753,1.807868,2.488324,1.968561,4.067363,2.627677,7.567024,5.331986,3.376342,2.038569,2.687443,5.476191,5.178084,2.47834,8.011064,3.543059,4.827703,3.017516,4.16355,3.221747,4.506296,8.235887,5.807355,26.724908,27.325342,27.609245,25.213966,24.331835,26.215639,22.81432,27.544977,22.19537,28.752434,20.419816,27.589608,27.468246,29.345561,27.630133,28.53968,24.170376,28.147612,28.736023,26.205761,27.696696,28.89871,24.924688,28.247088,28.323087,28.010072,30.00084,28.842624,28.864334,29.342918,28.687128,28.448141,29.606282,26.495599,29.524206,28.541065,28.031097,21.767546,26.980395,GO:0005911; GO:0036064; GO:0005737; GO:0005829; GO:0015630; GO:0005739; GO:0005654; GO:0005634; GO:0005886; GO:0043234; GO:0005819; GO:0031982; GO:0071889; GO:0005524; GO:0032794; GO:0042802; GO:0016301; GO:0030235; GO:0005547; GO:0043325; GO:0042803; GO:0004672; GO:0019901; GO:0005080; GO:0051721; GO:0004674; GO:0004712; GO:0004713; GO:0006924; GO:0007568; GO:0008637; GO:0030030; GO:0071276; GO:0036294; GO:0006974; GO:0071364; GO:0097011; GO:0071363; GO:0071456; GO:0032869; GO:0071260; GO:1990090; GO:0071407; GO:1901653; GO:0071380; GO:0034614; GO:0071356; GO:0035924; GO:0007010; GO:0007173; GO:0072655; GO:0097194; GO:0007281; GO:0042593; GO:0006006; GO:0015758; GO:0005978; GO:0060709; GO:0005977; GO:0030212; GO:0007249; GO:0006954; GO:0008286; GO:0048009; GO:0035655; GO:0035556; GO:0060716; GO:0031663; GO:0072656; GO:0001893; GO:0043066; GO:0010507; GO:0045792; GO:0043154; GO:0010951; GO:0031999; GO:0010629; GO:0046329; GO:0010748; GO:0006469; GO:0045861; GO:0090201; GO:0038061; GO:0001649; GO:0018105; GO:0018107; GO:0032287; GO:0014065; GO:0016310; GO:0043065; GO:0043536; GO:0030307; GO:0008284; GO:0031659; GO:0051091; GO:0032079; GO:0001938; GO:0045600; GO:0010763; GO:0046326; GO:0010907; GO:0045725; GO:1903721; GO:0046889; GO:0010918; GO:0045429; GO:0051000; GO:0046622; GO:0033138; GO:0032436; GO:1900182; GO:1903078; GO:0001934; GO:0048661; GO:0010765; GO:0045944; GO:0045893; GO:0045907; GO:0030163; GO:0000060; GO:0043491; GO:0006468; GO:0016567; GO:0042981; GO:0030334; GO:0005979; GO:0031641; GO:0010975; GO:0032880; GO:0006417; GO:0034405; GO:0032094; GO:0060416; GO:0009725; GO:1990418; GO:0010033; GO:0006979; GO:0070141; GO:0021510; GO:0051146; GO:0006412
ENSMUST00000002710,1.152644,1.769748,1.38618,0.88323,1.238459,1.818865,1.393137,2.055142,1.856314,1.522474,1.307268,1.082604,2.087398,2.186628,1.637338,3.160934,1.888891,1.859546,1.663773,0.869776,1.317399,1.477447,2.334978,1.540329,5.159958,3.762266,4.672721,4.813167,4.052656,4.540973,3.417692,4.62757,3.978401,3.930666,3.351347,4.630105,3.636565,4.485649,3.762605,4.626774,4.575903,4.581032,4.715888,4.280947,4.084651,4.51666,3.345868,4.220935,3.820685,4.019916,4.571506,3.860165,4.188177,5.447177,4.081221,4.931568,5.244935,3.727492,4.81019,4.327296,4.479633,3.52365,3.714444,GO:0005737; GO:0007049
ENSMUST00000004686; ENSMUST00000111081,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,GO:0005779; GO:0031231; GO:

In [141]:
import org.apache.spark.sql.catalyst.encoders.RowEncoder
def groupByGO(df: DataFrame) =
  {
    implicit val encoder = RowEncoder(df.schema)  
    df.flatMap{
      case row =>
        val g = row.getAs[String]("go")
        val goes = if(g==null) Array[String]() else g.split(";").map(go=>go.trim).filter(go=>go!="")
        val seq = row.toSeq
        goes.map{ go => Row.fromSeq(seq.map(s=> if(s==g) go else s))
        }
    }.groupBy("go").sum(mc.columns.tail:_*).persist(StorageLevel.MEMORY_AND_DISK) 
  }

groupByGO: (df: org.apache.spark.sql.DataFrame)org.apache.spark.sql.Dataset[org.apache.spark.sql.Row]


In [144]:
val renamingsGO = renamings.map{ case (key, value) => (s"sum($key)", value)}

renamingsGO = Map(sum(GSM1944863) -> skin_GSM1944863, sum(GSM1944785) -> blood_GSM1944785, sum(GSM1944812) -> brain_GSM1944812, sum(GSM1944817) -> brain_GSM1944817, sum(GSM1944768) -> blood_GSM1944768, sum(GSM1944857) -> skin_GSM1944857, sum(GSM1944823) -> brain_GSM1944823, sum(GSM2795211) -> brain_GSM2795211, sum(GSM1944775) -> blood_GSM1944775, sum(GSM1944866) -> skin_GSM1944866, sum(GSM1944831) -> liver_GSM1944831, sum(GSM1944853) -> liver_GSM1944853, sum(GSM1944790) -> blood_GSM1944790, sum(GSM1944797) -> brain_GSM1944797, sum(GSM1944839) -> liver_GSM1944839, sum(GSM1944822) -> brain_GSM1944822, sum(GSM1944794) -> brain_GSM1944794, sum(GSM1944825) -> brain_GSM1944825, sum(GSM1944778) -> blood_GSM1944778, sum(GSM1944844) -> liver_GSM1944...


Map(sum(GSM1944863) -> skin_GSM1944863, sum(GSM1944785) -> blood_GSM1944785, sum(GSM1944812) -> brain_GSM1944812, sum(GSM1944817) -> brain_GSM1944817, sum(GSM1944768) -> blood_GSM1944768, sum(GSM1944857) -> skin_GSM1944857, sum(GSM1944823) -> brain_GSM1944823, sum(GSM2795211) -> brain_GSM2795211, sum(GSM1944775) -> blood_GSM1944775, sum(GSM1944866) -> skin_GSM1944866, sum(GSM1944831) -> liver_GSM1944831, sum(GSM1944853) -> liver_GSM1944853, sum(GSM1944790) -> blood_GSM1944790, sum(GSM1944797) -> brain_GSM1944797, sum(GSM1944839) -> liver_GSM1944839, sum(GSM1944822) -> brain_GSM1944822, sum(GSM1944794) -> brain_GSM1944794, sum(GSM1944825) -> brain_GSM1944825, sum(GSM1944778) -> blood_GSM1944778, sum(GSM1944844) -> liver_GSM1944844, sum(GSM1944847) -> liver_GSM1944847, sum(GSM1944870) -> skin_GSM1944870, sum(GSM1944807) -> brain_GSM1944807, sum(GSM1944803) -> brain_GSM1944803, sum(GSM1944792) -> brain_GSM1944792, sum(GSM1944793) -> brain_GSM1944793, sum(GSM1944788) -> blood_GSM1944788, s

In [145]:
val gMice = rename(groupByGO(miceWithGO), renamingsGO)

gMice = [go: string, blood_GSM1944768: double ... 62 more fields]


[go: string, blood_GSM1944768: double ... 62 more fields]

In [146]:
gMice.count

17419

In [150]:
val mouseCorSpearmanGO = spearmanCorrellation(gMice, gMice.columns.tail)(spark)
    .withColumn("feature", renameFunction(renamingsGO)($"feature"))

val mouseCorPearsonGO  = pearsonCorrellation(gMice, gMice.columns.tail)(spark)
.withColumn("feature", renameFunction(renamingsGO)($"feature"))

mouseCorSpearmanGO = [feature: string, blood_GSM1944768: double ... 62 more fields]
mouseCorPearsonGO = [feature: string, blood_GSM1944768: double ... 62 more fields]


[feature: string, blood_GSM1944768: double ... 62 more fields]

In [153]:
%%dataframe mouseCorSpearmanGO

feature,blood_GSM1944768,blood_GSM1944769,blood_GSM1944770,blood_GSM1944771,blood_GSM1944772,blood_GSM1944773,blood_GSM1944774,blood_GSM1944775,blood_GSM1944776,blood_GSM1944777,blood_GSM1944778,blood_GSM1944779,blood_GSM1944780,blood_GSM1944781,blood_GSM1944782,blood_GSM1944783,blood_GSM1944784,blood_GSM1944785,blood_GSM1944786,blood_GSM1944787,blood_GSM1944788,blood_GSM1944789,blood_GSM1944790,blood_GSM1944791,brain_GSM1944792,brain_GSM1944793,brain_GSM1944794,brain_GSM1944795,brain_GSM1944796,brain_GSM1944797,brain_GSM1944798,brain_GSM1944799,brain_GSM1944800,brain_GSM1944801,brain_GSM1944802,brain_GSM1944803,brain_GSM1944804,brain_GSM1944805,brain_GSM1944806,brain_GSM1944807,brain_GSM1944808,brain_GSM1944809,brain_GSM1944810,brain_GSM1944811,brain_GSM1944812,brain_GSM1944813,brain_GSM1944814,brain_GSM1944815,brain_GSM1944816,brain_GSM1944817,brain_GSM1944818,brain_GSM1944819,brain_GSM1944820,brain_GSM1944821,brain_GSM1944822,brain_GSM1944823,brain_GSM1944824,brain_GSM1944825,brain_GSM1944826,brain_GSM1944827,brain_GSM1944828,brain_GSM1944829,brain_GSM1944830
blood_GSM1944768,1.0,0.9877416747805284,0.9883839303357295,0.9837945224703988,0.9880536855095976,0.9848433540141376,0.983604301986465,0.9783286238717319,0.9830085160614356,0.9828573470404884,0.985617629118902,0.9825438489167699,0.9830699766903279,0.9836375271623312,0.9854077190774355,0.9816808111687929,0.9845973812064179,0.9781552492955442,0.9834098088257707,0.9768295515457427,0.9809427263508503,0.9795884549568258,0.9790189411994663,0.9795009217892676,0.8134252135439506,0.8009136175960636,0.8004775849896081,0.8116333080127627,0.8155067579068798,0.8049407395467647,0.8181966038468244,0.8047806304607543,0.8223604204396866,0.803848912600726,0.8258351233331401,0.8030409672535882,0.8135742333461246,0.8047395428548025,0.8150085410343606,0.8063109296295122,0.8169577008242882,0.8082318032566714,0.8056401674161431,0.8040665494307135,0.8134391626571471,0.8059007941369308,0.8154899699051452,0.8078636238314912,0.8079639902270077,0.8049431488529916,0.8015412629578084,0.8141093287882343,0.8112712016284775,0.8047770366283109,0.8128243164162874,0.8215194361886706,0.8067782998158946,0.8231855813516971,0.8074976937455273,0.8072835010477432,0.8065274278636534,0.8423571118732367,0.8199331489892943
blood_GSM1944769,0.9877416747805284,1.0,0.9897855645829982,0.9863693632539611,0.9892259799503949,0.9876502947148946,0.9849946255432092,0.9833220236709906,0.9879374663314192,0.9873862437502897,0.9885543323080419,0.9841634054918789,0.9874119693027908,0.9880536687518152,0.9864993798137057,0.9868830844650426,0.987284197251538,0.9812907622976694,0.9868975811983052,0.9785472916037278,0.9816622151507401,0.9823575105880648,0.9825486843264377,0.9828461005385404,0.8195711035588799,0.807476853306257,0.8070994424087482,0.8185445638787611,0.8217744366328618,0.811156067041565,0.824300505347253,0.8112579926707167,0.8288158122929472,0.8104983863468078,0.8320351868104049,0.8096753627814373,0.8201985697653866,0.8115402044874014,0.8216712497060169,0.8131760634236924,0.8231201829361047,0.8148082035934885,0.8125768539171396,0.8106666036571909,0.819867214618322,0.8125249915526357,0.8217386123508249,0.8143306506116512,0.8142614705973866,0.8116602723819798,0.8084262416518866,0.8204710200992472,0.8180583394596185,0.8114618464981339,0.8196273033450518,0.8278949290949578,0.813519550775271,0.8293470909768569,0.8142136228813228,0.813990235719355,0.8132912420806393,0.8484016902670617,0.8263714618385711
blood_GSM1944770,0.9883839303357295,0.9897855645829982,1.0,0.9872553360784726,0.9898060834294602,0.9867287037551967,0.9858069365893185,0.9830583142088147,0.9866936770975184,0.9872249731348723,0.9882430609916458,0.986366704116296,0.9863166431841717,0.9875622991826515,0.9873942109789802,0.985586298021317,0.9870209225028074,0.981798167847199,0.9870245335349136,0.9803150643633236,0.9829571631092803,0.9827037251174621,0.9816451636195178,0.982337424417721,0.815250556363395,0.8029561016162314,0.8024799816109384,0.8138079506265583,0.81

In [154]:
%%dataframe mouseCorPearsonGO

feature,blood_GSM1944768,blood_GSM1944769,blood_GSM1944770,blood_GSM1944771,blood_GSM1944772,blood_GSM1944773,blood_GSM1944774,blood_GSM1944775,blood_GSM1944776,blood_GSM1944777,blood_GSM1944778,blood_GSM1944779,blood_GSM1944780,blood_GSM1944781,blood_GSM1944782,blood_GSM1944783,blood_GSM1944784,blood_GSM1944785,blood_GSM1944786,blood_GSM1944787,blood_GSM1944788,blood_GSM1944789,blood_GSM1944790,blood_GSM1944791,brain_GSM1944792,brain_GSM1944793,brain_GSM1944794,brain_GSM1944795,brain_GSM1944796,brain_GSM1944797,brain_GSM1944798,brain_GSM1944799,brain_GSM1944800,brain_GSM1944801,brain_GSM1944802,brain_GSM1944803,brain_GSM1944804,brain_GSM1944805,brain_GSM1944806,brain_GSM1944807,brain_GSM1944808,brain_GSM1944809,brain_GSM1944810,brain_GSM1944811,brain_GSM1944812,brain_GSM1944813,brain_GSM1944814,brain_GSM1944815,brain_GSM1944816,brain_GSM1944817,brain_GSM1944818,brain_GSM1944819,brain_GSM1944820,brain_GSM1944821,brain_GSM1944822,brain_GSM1944823,brain_GSM1944824,brain_GSM1944825,brain_GSM1944826,brain_GSM1944827,brain_GSM1944828,brain_GSM1944829,brain_GSM1944830
blood_GSM1944768,1.0,0.9846446483622926,0.9980851371766833,0.9917084292717487,0.9971568775071147,0.988064636071325,0.9893788180748317,0.9744811513775106,0.9896245485655729,0.9855079310052055,0.9973992458285029,0.9853886300946268,0.9910970661815564,0.9933906412903273,0.9947226206389634,0.9604928670034798,0.9643237296879814,0.9775477705374063,0.9928963163285105,0.9581407924217994,0.9747148173111069,0.982816481104321,0.9639552147250152,0.9863398737361496,0.7047498046388438,0.7168118730950348,0.7172034922873363,0.7218648945157745,0.7082100185348961,0.7110356062151517,0.7026150661135709,0.7193843155134402,0.7100762524958609,0.7148190328632098,0.6969602040757783,0.7171290844308347,0.7113712056195775,0.7201269564949477,0.7045860439593817,0.717643993918895,0.7077783923145217,0.7183304615548879,0.7127728234017252,0.7108844765654995,0.7118886789439702,0.7144170439882366,0.6963932325549816,0.7144223068341461,0.7095067258555272,0.7170258176397171,0.720688652554987,0.7153314294026786,0.7243735869955589,0.7187814242083671,0.7121861045053898,0.7186896003098009,0.7178896291770689,0.7116994318970996,0.7243179605752254,0.7210633874194229,0.7153651704795391,0.7174431790106984,0.7204653185313988
blood_GSM1944769,0.9846446483622926,1.0,0.9886882730623747,0.9652970776684366,0.9902194662876374,0.9917285595399368,0.9907421428188578,0.9489227412901139,0.9823891021991021,0.9519979660317458,0.9872921894534505,0.9492855262878593,0.9862774310811374,0.9860633915751444,0.9781091143174531,0.9886177636472916,0.9906561389704172,0.9669839125706404,0.9849314014700292,0.9063173026750321,0.9736857229291959,0.9912562370427779,0.9798439430954868,0.961651897846422,0.76227192569714,0.7709336828624928,0.7709508917048264,0.7789731405910428,0.7655538684631256,0.7668716017091509,0.7627951035845694,0.7748693556626918,0.7710879257489398,0.7692944212781417,0.7580632566583967,0.7712193922143836,0.7681095860114069,0.774339233880614,0.7626648383149195,0.7731243660008059,0.7664041097427089,0.7732634265147612,0.7674394100707451,0.7657484854457492,0.7687385454854556,0.7694331852111707,0.7550277346050888,0.7703268709896878,0.764954179673366,0.7716512959173295,0.7739895384750263,0.7721796660245356,0.7794180798658052,0.772785185173507,0.7689059975456449,0.7759851642655538,0.7721807178865634,0.7704231270266758,0.7781723021873744,0.7750004642749844,0.7699636449102923,0.7790261332278514,0.7776630784191864
blood_GSM1944770,0.9980851371766833,0.9886882730623747,1.0,0.9890540229150302,0.9975569435246735,0.9892708047917383,0.9906453863076788,0.9730204497211967,0.9903135014614604,0.9823258433612965,0.9980388475565418,0.9792135261614832,0.9952516172138786,0.9952694832112665,0.9915513038470141,0.9670166833563026,0.9668418002173436,0.978238467330825,0.991198768930123,0.9496243679729596,0.975800531273551,0.9881819147354685,0.9723021542333821,0.9831951992606196,0.693726082365405,0.7063333900588964,0.7067419246243296,0.7115517835133314,0.6

In [155]:
write(mouseCorSpearmanGO, az(corPath + "/spearman_go_mouse_GSE75192.tsv"))
write(mouseCorPearsonGO, az(corPath + "/pearson_go_mouse_GSE75192.tsv"))

wasbs://storage@pipelines1.blob.core.windows.net/expressions/go/grouped/correlations/pearson_go_mouse_GSE75192.tsv

In [156]:
write(gMice, az(jenageValidationPath + "/go_mouse_GSE75192.tsv"))

wasbs://storage@pipelines1.blob.core.windows.net/expressions/validation/mouse/GSE75192/go_mouse_GSE75192.tsv

In [2]:
%AddJar -magic https://brunelvis.org/jar/spark-kernel-brunel-all-2.6.jar

Starting download from https://brunelvis.org/jar/spark-kernel-brunel-all-2.6.jar
Finished download of spark-kernel-brunel-all-2.6.jar


In [3]:
 %%brunel data('mouseCorPearsonGO') map x(feature)

<console>:158: error: not found: value mouseCorPearsonGO
       mouseCorPearsonGO
       ^


Magic brunel failed to execute with error: 
compile-time error

For Mitya
=====

Two cells below are for Mitya to get infromation about Uniref90 and Go. As

In [34]:
val go = "GO:0070062" //put your GO term inside quotes
val searchGO = mapping.filter($"go".contains(go))

go = GO:0070062
searchGO = [uniprot_ac: string, uniprot_id: string ... 20 more fields]


[uniprot_ac: string, uniprot_id: string ... 20 more fields]

In [35]:
%%dataframe searchGO

uniprot_ac,uniprot_id,entrez,refSeq,gi,pdb,go,uniref100,uniref90,uniref50,uniparc,pir,taxon,mim,unigene,pubmed,embl,embl_cds,ensembl,ensembl_trs,ensembl_pro,additional_pubmed
Q5ZLQ6,1433B_CHICK,419190,NP_001006289.1,53128824; 82197872; 57529350,null,GO:0005829; GO:0070062; GO:0005925; GO:0016020; GO:0048471; GO:0045296; GO:0042826; GO:0042802; GO:0050815; GO:0019904; GO:0051220; GO:0045744; GO:0035308; GO:0043085; GO:0006605,UniRef100_Q5ZLQ6,UniRef90_Q5ZLQ6,UniRef50_P61981,UPI00004434D0,null,9031,null,Gga.34332,15642098,AJ719678,CAG31337.1,ENSGALG00000004143,ENSGALT00000044506,ENSGALP00000043305,22174833
P31946,1433B_HUMAN,7529,NP_003395.1; NP_647539.1; XP_016883528.1,4507949; 377656702; 67464628; 1345590; 1034625756; 21328448; 377656701; 67464627; 78101741,2BQ0:A; 2BQ0:B; 2C23:A; 4DNK:A; 4DNK:B; 5N10:A; 5N10:B,GO:0005737; GO:0030659; GO:0005829; GO:0070062; GO:0005925; GO:0042470; GO:0016020; GO:0005739; GO:0005634; GO:0048471; GO:0043234; GO:0017053; GO:0045296; GO:0019899; GO:0042826; GO:0042802; GO:0051219; GO:0050815; GO:0008022; GO:0032403; GO:0019904; GO:0003714; GO:0051220; GO:0035329; GO:0000165; GO:0061024; GO:0045744; GO:0035308; GO:0045892; GO:0043085; GO:1900740; GO:0051291; GO:0006605; GO:0043488; GO:0016032,UniRef100_P31946,UniRef90_P31946,UniRef50_P31946,UPI000013C714,S34755,9606,601289,Hs.643544,8515476; 14702039; 11780052; 15489334; 23572552; 12665801; 11427721; 15454081; 15538381; 15159416; 15696159; 17081065; 17974916; 17717073; 18249187; 19172738; 19640509; 19592491; 19690332; 19608861; 21269460; 21224381; 22827337; 22814378; 23186163; 23938468; 24636949; 24275569; 26047703; 25588844; 25489052; 25944712; 17085597; 21248752,X57346; AK292717; AL008725; CH471077; CH471077; CH471077; BC001359,CAA40621.1; BAF85406.1; CAA15497.1; EAW75893.1; EAW75894.1; EAW75896.1; AAH01359.1,ENSG00000166913,ENST00000353703; ENST00000372839,ENSP00000300161; ENSP00000361930,11996670; 12364343; 12437930; 12468542; 12482592; 12582162; 12618428; 12669242; 14680818; 15022330; 15389601; 15726117; 16166738; 17098443; 17229891; 17531190; 18094049; 19173300; 19360691; 19558434; 19821490; 20070120; 20381070; 20388496; 21147917; 21262972; 21396404; 21553213; 21598387; 21708191; 21935479; 21948273; 21972092; 22125622; 22278744; 23053962; 24038028; 24269229; 24555778; 25228695; 26260846; 26730736; 28510303; 10409742; 10713667; 10775038; 10840038; 10862767; 10958686; 11157475; 11279064; 11313964; 11555644; 11697890; 11886850; 12383250; 12438239; 12620389; 12871587; 12963375; 14651979; 14679215; 14688255; 14701738; 14743216; 15023544; 15057270; 15173315; 15324660; 15629149; 15684389; 15691829; 15860732; 15883195; 16282323; 16511560; 16581770; 16672054; 16672277; 16717153; 16775625; 16868027; 16959763; 17170118; 17314511; 17353931; 17461779; 17500595; 17620599; 17923693; 17965023; 17979178; 18029035; 18160719; 18308725; 18372248; 18458160; 18460465; 18626018; 18647389; 18771726; 19129461; 19135240; 19270694; 19329994; 19595761; 19615732; 19691494; 19725029; 19738201; 19805454; 19860830; 19955570; 20000738; 20048001; 20169078; 20195357; 20363754; 20562859; 20598904; 20618440; 20639859; 20642453; 20936779; 21112954; 21278420; 21282530; 21565611; 21706016; 21771788; 21900206; 21903422; 21911578; 22078878; 22190034; 22304920; 22593156; 22623428; 22677168; 22810585; 23602568; 23622247; 23752268; 24255178; 24366813; 24510904; 24658140; 24947832; 25036637; 25277244; 25609649; 25852190; 26496610; 26514267; 26638075; 7644510; 7882972; 7935795; 8085158; 9341175; 9581554
Q9CQV8,1433B_MOUSE,54401,NP_061223.2; XP_006499972.1,1009083986; 409974018; 74219534; 31543974; 3065925; 568917832; 18202836; 74191394,4GNT:A; 5F74:A,GO:0005737; GO:0030659; GO:0005829; GO:0070062; GO:0005925; GO:0042470; GO:0016020; GO:0005634; GO:0048471; GO:0043234; GO:0017053; GO:0045296; GO:0019899; GO:0042826; GO:0042802; GO:0051219; GO:0050815; GO:0008022; GO:0032403; GO:0019904; GO:0003714; GO:0051220; GO:0045744; GO:0035308; GO:0045892; GO:0043085; GO:0051291; GO:0006605,UniRef10

In [21]:
val uniref90: String = "UniRef90_Q6GZX4" //put your Uniref90 term inside quotes
val searchUniref90 = mapping.filter($"uniref90".contains(uniref90))

uniref90 = UniRef90_Q6GZX4
searchUniref90 = [uniprot_ac: string, uniprot_id: string ... 20 more fields]


[uniprot_ac: string, uniprot_id: string ... 20 more fields]

In [5]:
%%dataframe searchUniref90

uniprot_ac,uniprot_id,entrez,refSeq,gi,pdb,go,uniref100,uniref90,uniref50,uniparc,pir,taxon,mim,unigene,pubmed,embl,embl_cds,ensembl,ensembl_trs,ensembl_pro,additional_pubmed
Q6GZX4,001R_FRG3G,2947773,YP_031579.1,81941549; 49237298,null,GO:0006355; GO:0046782; GO:0006351,UniRef100_Q6GZX4,UniRef90_Q6GZX4,UniRef50_Q6GZX4,UPI00003B0FD4,null,654924,null,null,15165820,AY548484,AAT09660.1,null,null,null,null
